# Fantasy Premier League Management

In [41]:
from bs4 import BeautifulSoup
from collections import defaultdict
from fuzzywuzzy import process, fuzz  # For fuzzy string matching
import openpyxl
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe"
from pulp import *
import random
import re
import requests
import unidecode  # Handles removal of accents/diacritics
import unicodedata

# Suppress the SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

## Helper Functions

In [42]:
def calculate_average_fdr(team, position, fdr_data, gameweeks_ahead):
    """Helper function to calculate average FDR for a player's team."""
    team_fixtures = fdr_data[fdr_data['Home Team'] == team]
    team_fixtures = team_fixtures.head(gameweeks_ahead)  # Limit to upcoming fixtures
    if not team_fixtures.empty:
        # Calculate average FDR
        return team_fixtures['Home Team Difficulty'].mean()
    return 5.0  # Default high FDR if no fixtures found

In [43]:
def normalize_apostrophes(text):
    """ Normalize different types of apostrophes to a standard single quote """
    return text.replace("’", "'").replace("`", "'")

In [44]:
def remove_duplicate_words(name):
    """
    Function to remove duplicate consecutive words
    """
    return re.sub(r'\b(\w+)\s+\1\b', r'\1', name)

In [45]:
def normalize_name(name):
    """
    Normalize a player name by:
    1. Converting it to ASCII (removing accents/diacritics).
    2. Lowercasing the name.
    3. Stripping extra spaces and symbols.
    4. Keeping only the first and last name (ignoring middle names).
    
    Parameters:
    - name (str): The player name to normalize.
    
    Returns:
    - str: The normalized player name.
    """
    # Remove accents/diacritics and lowercase the name
    normalized_name = unidecode.unidecode(name).lower()

    # Split the name and keep only the first and last parts
    name_parts = normalized_name.split()
    if len(name_parts) >= 2:
        normalized_name = f"{name_parts[0]} {name_parts[-1]}"
    else:
        normalized_name = name_parts[0]

    return normalized_name

In [46]:
def check_valid_lineup(df):
    """
    Given a dataframe with a lineup, check to see if it is a valid lineup.
    Requirements:
    - 11 total players
    - 1 GK
    - Min of 3 DEF
    - Max of 5 DEF
    - Min of 3 MID
    - Max of 5 MID
    - Min of 1 FWD
    - Max of 3 MID
    """
    # Check the total players count
    players = len(df)

    # Count occurrences of each value in the 'Position' column
    position_counts = df['position'].value_counts()

    # Perform the checks
    player_check = players == 11
    gk_check = position_counts['G'] == 1
    def_check = position_counts['D'] >= 3 and position_counts['D'] <= 5
    mid_check = position_counts['M'] >= 3 and position_counts['M'] <= 5
    fwd_check = position_counts['F'] >= 1 and position_counts['F'] <= 3

    # Lineup is valid is all checks are true
    return (player_check & gk_check & def_check & mid_check & fwd_check)

In [47]:
def clean_fpl_player_names(fpl_players_df, rotowire_projections_df, fuzzy_threshold=80, lower_fuzzy_threshold=60):
    """
    Cleans the player names in the FPL DataFrame by replacing them with their best matches from the projections DataFrame.

    Parameters:
    - fpl_players_df: DataFrame with FPL players ['Player', 'Team', 'Position'].
    - rotowire_projections_df: DataFrame with Rotowire projections ['Player', 'Team', 'Position'].
    - fuzzy_threshold: Default fuzzy matching threshold for player names.
    - lower_fuzzy_threshold: Lower threshold if team and position match.

    Returns:
    - fpl_players_df: Updated FPL DataFrame with cleaned player names.
    """
    
    def find_best_match(fpl_player, fpl_team, fpl_position, candidates):
        """Finds the best match for a player using fuzzy matching."""
        match, score = process.extractOne(fpl_player, candidates)

        matched_row = rotowire_projections_df[rotowire_projections_df['Player'] == match]
        if not matched_row.empty:
            match_team = matched_row.iloc[0]['Team']
            match_position = matched_row.iloc[0]['Position']

            if match_team == fpl_team and match_position == fpl_position and score >= lower_fuzzy_threshold:
                return match

        if score >= fuzzy_threshold:
            return match

        return fpl_player  # Return original name if no good match is found

    # Extract candidate names from projections
    projection_names = rotowire_projections_df['Player'].tolist()

    # Update FPL DataFrame with cleaned player names
    fpl_players_df['Player'] = fpl_players_df.apply(
        lambda row: find_best_match(row['Player'], row['Team'], row['Position'], projection_names), axis=1
    )

    return fpl_players_df

In [48]:
def get_current_gameweek():
    """
    Fetches the current gameweek based on the game status from the FPL Draft API.

    Returns:
    - current_gameweek: An integer representing the current gameweek.
    """
    game_url = "https://draft.premierleague.com/api/game"
    response = requests.get(game_url)
    game_data = response.json()

    # Check if the current event is finished
    if game_data['current_event_finished']:
        current_gameweek = game_data['next_event']
    else:
        current_gameweek = game_data['current_event']

    return current_gameweek

In [49]:
def get_fixture_difficulty():
    """
    Fetches fixture difficulty data for all teams from the FPL API.

    Returns:
    - DataFrame containing structured fixture difficulty information.
    """
    # Endpoint to fetch fixture data
    fixtures_url = "https://draft.premierleague.com/api/bootstrap-static"

    # Fetch data from the API
    response = requests.get(fixtures_url)
    response.raise_for_status()
    data = response.json()

    # Extract relevant sections
    if 'fixtures' not in data or 'teams' not in data:
        raise ValueError("Unexpected API response structure. Missing 'fixtures' or 'teams'.")

    fixtures = data['fixtures']
    teams = {team['id']: team['short_name'] for team in data['teams']}

    # List to hold structured fixture data
    fixtures_data = []

    # Iterate through gameweeks and fixtures
    for gameweek, gameweek_fixtures in fixtures.items():
        for fixture in gameweek_fixtures:
            # Extract relevant fixture details
            fixtures_data.append({
                "Gameweek": int(gameweek),
                "Home Team": teams.get(fixture['team_h'], f"Unknown ({fixture['team_h']})"),
                "Away Team": teams.get(fixture['team_a'], f"Unknown ({fixture['team_a']})"),
                "Kickoff Time": fixture.get('kickoff_time'),
                "Home Team Difficulty": fixture.get('team_h_difficulty', "N/A"),
                "Away Team Difficulty": fixture.get('team_a_difficulty', "N/A")
            })

    # Convert to a DataFrame
    fixtures_df = pd.DataFrame(fixtures_data)

    # Sort by gameweek and kickoff time for better readability
    fixtures_df.sort_values(by=['Gameweek', 'Kickoff Time'], inplace=True)

    return fixtures_df

In [50]:
def find_optimal_lineup(df):
    """
    Function to find a team's optimal lineup given their player_projections_df
    :param df: a dataframe of the team's player projections
    :return:
    """
    # 1. Find the top scoring GK
    top_gk = df[df['position'] == 'G'].sort_values(by='points', ascending=False).head(1)

    # 2. Find the top 3 scoring DEF
    top_def = df[df['position'] == 'D'].sort_values(by='points', ascending=False).head(3)

    # 3. Find the top 3 scoring MID
    top_mid = df[df['position'] == 'M'].sort_values(by='points', ascending=False).head(3)

    # 4. Find the top scoring FWD
    top_fwd = df[df['position'] == 'F'].sort_values(by='points', ascending=False).head(1)

    # 5. Combine the selected players
    selected_players = pd.concat([top_gk, top_def, top_mid, top_fwd])

    # 6. Find the remaining top 3 scoring players not in the selected players
    remaining_players = df[~df['player'].isin(selected_players['player'])]
    top_remaining = remaining_players.sort_values(by='points', ascending=False).head(3)

    # 7. Combine all selected players
    final_selection = pd.concat([selected_players, top_remaining])

    # 8. Organize the final selection by Position and descending Projected_Points
    final_selection = final_selection.sort_values(
        by=['position', 'points'],
        key=lambda x: x.map({'G': 0, 'D': 1, 'M': 2, 'F': 3}),
        ascending=[True, False]
    ).reset_index(drop=True)

    # Check if valid lineup
    if check_valid_lineup(final_selection):
        # Display the final selection
        return(final_selection)
    else:
        print("Invalid Lineup")
        return(final_selection)

In [51]:
def pull_fpl_player_stats():

    # Set FPL Draft API endpoint
    draft_api = 'https://draft.premierleague.com/api/bootstrap-static'

    # Test the endpoint
    data = requests.get(draft_api)

    # extracting data in json format
    data_json = data.json()

    # Create a dataframe for the positions ('element_types')
    position_df = pd.DataFrame.from_records(data_json['element_types'])

    # Format df
    cols = ['id', 'singular_name', 'singular_name_short']
    position_df = position_df[cols]

    # Rename columns
    position_df.columns = ['position_id', 'position_name', 'position_abbrv']

    # Create a dataframe for the teams
    team_df = pd.DataFrame.from_records(data_json['teams'])

    # Format df
    cols = ['id', 'name', 'short_name']
    team_df = team_df[cols]

    # Rename columns
    team_df.columns = ['team_id', 'team_name', 'team_name_abbrv']

    # Create a DataFrame from the Player dictionary ('elements')
    player_stats_df = pd.DataFrame.from_records(data_json['elements'])

    # Create Full Name Column
    player_stats_df['player'] = player_stats_df['first_name'] + ' ' + player_stats_df['second_name']
    # Apply the remove_duplicate_words function to the 'Player' column
    player_stats_df['player'] = player_stats_df['player'].apply(remove_duplicate_words)

    # Merge in team_name
    player_stats_df = pd.merge(player_stats_df, team_df, left_on='team', right_on='team_id')

    # Merge in position name
    player_stats_df = pd.merge(player_stats_df, position_df, left_on='element_type', right_on='position_id')

    # Organize columns
    cols = ['id', 'player', 'position_abbrv', 'team_name', 'clean_sheets', 'goals_scored', 
            'assists', 'minutes', 'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'yellow_cards', 
            'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded',
            'creativity', 'influence', 'bonus', 'bps', 'form', 'points_per_game', 'total_points',
            'corners_and_indirect_freekicks_order', 'corners_and_indirect_freekicks_text', 'direct_freekicks_order',
            'direct_freekicks_text', 'penalties_order', 'penalties_text', 'chance_of_playing_this_round', 
            'chance_of_playing_next_round', 'status', 'added']
    player_stats_df = player_stats_df[cols]
    
    # Ensure expected_goal_involvements is numeric
    player_stats_df["expected_goal_involvements"] = pd.to_numeric(
        player_stats_df["expected_goal_involvements"], errors="coerce"
    )

    # Create actual goal involvements column and ensure that it is numeric (sum of goals + assists)
    player_stats_df["actual_goal_involvements"] = pd.to_numeric(
        player_stats_df["goals_scored"], errors="coerce"
    ) + pd.to_numeric(player_stats_df["assists"], errors="coerce")

    # Make sure total_points is numeric
    player_stats_df['total_points'] = pd.to_numeric(player_stats_df['total_points'], errors='coerce').fillna(0)

    # Sort dataframe by total_points
    player_stats_df = player_stats_df.sort_values(by='total_points', ascending = False)

    # Return df
    return(player_df)

In [52]:
def get_rotowire_rankings_url():
    """
    Fetches the full article URL for the latest Fantasy Premier League Player Rankings.

    Returns:
    - str: The full URL to the latest Rotowire player rankings article, or None if not found.
    """
    try:
        # Rotowire soccer homepage
        base_url = "https://www.rotowire.com"
        soccer_url = base_url + "/soccer/"
        response = requests.get(soccer_url)
        response.raise_for_status()  # Ensure the request was successful

        soup = BeautifulSoup(response.content, 'html.parser')

        # Get the current gameweek from your helper function
        current_gameweek = get_current_gameweek()

        # Article title to search for
        target_title = f"Fantasy Premier League Player Rankings: Gameweek {current_gameweek}"

        # Find all compact article sections
        articles = soup.find_all('div', class_='compact-article__main')

        # Iterate through the articles to find the matching one
        for article in articles:
            link_tag = article.find('a', class_='compact-article__link')
            if link_tag and target_title in link_tag.text:
                # Construct the correct URL without double '/soccer/' issue
                article_url = base_url + link_tag['href']
                return article_url

        print(f"No article found for Gameweek {current_gameweek}.")
        return None  # Explicitly return None if no matching article is found

    except Exception as e:
        print(f"Error fetching the Rotowire rankings URL: {e}")
        return None

In [53]:
def get_rotowire_player_projections(url, limit=None):
    """
    Fetches fantasy rankings and projected points for players from RotoWire.

    Parameters:
    - url (str): URL to fetch the data from.
    - limit (int, optional): Number of players to display. Defaults to None (displays all players).

    Returns:
    - DataFrame: A DataFrame containing player rankings, projected points, and calculated value.
    """
    # Download the page using the requests library
    website = requests.get(url)
    soup = BeautifulSoup(website.content, 'html.parser')

    # Isolate BeautifulSoup output to the table of interest
    my_classes = soup.find(class_='article-table__tablesorter article-table__standard article-table__figure')
    players = my_classes.find_all("td")

    # Create lists for each field to collect
    overall_rank, fw_rank, mid_rank, def_rank, gk_rank = [], [], [], [], []
    player, team, matchup, position, price, tsb, points = [], [], [], [], [], [], []

    # Loop through the list of players in batches of 12
    batch_size = 12
    for i in range(0, len(players), batch_size):
        overall_rank.append(players[i].text)
        fw_rank.append(players[i + 1].text)
        mid_rank.append(players[i + 2].text)
        def_rank.append(players[i + 3].text)
        gk_rank.append(players[i + 4].text)
        player.append(players[i + 5].text)
        team.append(players[i + 6].text)
        matchup.append(players[i + 7].text)
        position.append(players[i + 8].text)
        price.append(players[i + 9].text)
        tsb.append(players[i + 10].text)
        points.append(players[i + 11].text)

    # Create a DataFrame with formatted column names
    player_rankings = pd.DataFrame(
        list(zip(overall_rank, fw_rank, mid_rank, def_rank, gk_rank, player, team,
                 matchup, position, price, tsb, points)),
        columns=[
            'Overall Rank', 'FW Rank', 'MID Rank', 'DEF Rank', 'GK Rank',
            'Player', 'Team', 'Matchup', 'Position', 'Price', 'TSB %', 'Points'
        ]
    )

    # Replace empty strings with 0 and convert columns to numeric where appropriate
    for col in ['FW Rank', 'MID Rank', 'DEF Rank', 'GK Rank', 'Points', 'Price']:
        player_rankings[col] = pd.to_numeric(player_rankings[col], errors='coerce').fillna(0)

    # Create 'Pos Rank' by summing the four position ranks
    player_rankings['Pos Rank'] = (
        player_rankings['FW Rank'] + player_rankings['MID Rank'] +
        player_rankings['DEF Rank'] + player_rankings['GK Rank']
    ).astype(int)

    # Drop individual position rank columns
    player_rankings.drop(columns=['FW Rank', 'MID Rank', 'DEF Rank', 'GK Rank'], inplace=True)

    # Ensure 'Price' is numeric
    player_rankings['Price'] = pd.to_numeric(player_rankings['Price'], errors='coerce').fillna(0)

    # Create the 'Value' column by dividing 'Points' by 'Price'
    player_rankings['Value'] = player_rankings.apply(
        lambda row: row['Points'] / row['Price'] if row['Price'] > 0 else float('nan'), axis=1
    )

    # If a limit is provided, return only the top 'limit' players
    if limit:
        player_rankings = player_rankings.head(limit)

    # Format the DataFrame to remove the index and reset it with a starting value of 1
    player_rankings.reset_index(drop=True, inplace=True)
    player_rankings.index = player_rankings.index + 1

    return player_rankings

In [54]:
def merge_fpl_and_projections(fpl_players_df, projections_df, fuzzy_threshold=80, lower_fuzzy_threshold=60):
    """
    Merges the FPL player data with Rotowire projections based on player name, team, and position.

    Parameters:
    - fpl_players_df: DataFrame with FPL players ['player', 'team', 'position'].
    - projections_df: DataFrame with Rotowire projections.
    - fuzzy_threshold: Default fuzzy matching threshold for player names.
    - lower_fuzzy_threshold: Lower threshold if team and position match.

    Returns:
    - merged_df: DataFrame with players, projections, and any missing players shown with NA values.
    """

    def find_best_match(fpl_player, fpl_team, fpl_position, candidates):
        """Finds the best match for a player using fuzzy matching."""
        match, score = process.extractOne(fpl_player, candidates)

        matched_row = projections_df[projections_df['player'] == match]
        if not matched_row.empty:
            match_team = matched_row.iloc[0]['team']
            match_position = matched_row.iloc[0]['position']

            if match_team == fpl_team and match_position == fpl_position and score >= lower_fuzzy_threshold:
                return match

        if score >= fuzzy_threshold:
            return match

        return None

    merged_data = []

    for _, fpl_row in fpl_players_df.iterrows():
        fpl_player = fpl_row['player']
        fpl_team = fpl_row['team']
        fpl_position = fpl_row['position']

        candidates = projections_df['player'].tolist()
        best_match = find_best_match(fpl_player, fpl_team, fpl_position, candidates)

        if best_match:
            matched_row = projections_df[projections_df['player'] == best_match].iloc[0]
            merged_data.append({
                'Player': matched_row['player'],
                'Team': matched_row['team'],
                'Matchup': matched_row.get('matchup', ''),
                'Position': matched_row['position'],
                'Price': matched_row.get('price', float('nan')),
                'TSB%': matched_row.get('tsb', float('nan')),
                'Points': matched_row['points'],
                'Pos Rank': matched_row.get('pos_rank', 'NA')
            })
        else:
            merged_data.append({
                'Player': fpl_player,
                'Team': fpl_team,
                'Matchup': 'N/A',
                'Position': fpl_position,
                'Price': float('nan'),
                'TSB%': float('nan'),
                'Points': float('nan'),
                'Pos Rank': 'NA'
            })

    # Create the DataFrame
    merged_df = pd.DataFrame(merged_data)

    # Reorder columns
    merged_df = merged_df[['Player', 'Team', 'Matchup', 'Position', 'Price', 'TSB%', 'Points', 'Pos Rank']]

    # Ensure 'Pos Rank' is integer or 'NA'
    merged_df['Pos Rank'] = pd.to_numeric(merged_df['Pos Rank'], errors='coerce').fillna(-1).astype(int)
    merged_df['Pos Rank'] = merged_df['Pos Rank'].replace(-1, 'NA')
    
    # Set index to represent overall rank, starting at 1
    merged_df.index = pd.RangeIndex(start=1, stop=len(merged_df) + 1, step=1)

    return merged_df

In [55]:
def analyze_fixture_projections(fixture, league_id, projections_df):
    """
    Returns two DataFrames representing optimal projected lineups and points for each team in a fixture,
    sorted by position (GK, DEF, MID, FWD) and then by descending projected points within each position.

    Parameters:
    - fixture (str): The selected fixture, formatted as "Team1 (Player1) vs Team2 (Player2)".
    - league_id (int): The ID of the FPL Draft league.
    - projections_df (DataFrame): DataFrame containing player projections from Rotowire.

    Returns:
    - Tuple of two DataFrames: (team1_df, team2_df, team1_name, team2_name)
    """
    # Normalize the apostrophes in the fixture string
    fixture = normalize_apostrophes(fixture)

    # Extract the team names only (ignore player names inside parentheses)
    team1_name = fixture.split(' vs ')[0].split(' (')[0].strip()
    team2_name = fixture.split(' vs ')[1].split(' (')[0].strip()

    # Get the current gameweek
    gameweek = get_current_gameweek()

    # Retrieve team compositions for the current gameweek and convert to dataframes
    team1_composition = get_team_composition_for_gameweek(league_id, team1_name, gameweek)
    team2_composition = get_team_composition_for_gameweek(league_id, team2_name, gameweek)

    # Merge FPL players with projections for both teams
    team1_df = merge_fpl_players_and_projections(
        team1_composition, projections_df[['Player', 'Team', 'Position', 'Matchup', 'Points', 'Pos Rank']]
    )
    team2_df = merge_fpl_players_and_projections(
        team2_composition, projections_df[['Player', 'Team', 'Position', 'Matchup', 'Points', 'Pos Rank']]
    )

    # Debugging: Check if 'Points' column exists
    if 'Points' not in team1_df or 'Points' not in team2_df:
        print("Error: 'Points' column not found in one or both dataframes.")
        print("Team 1 DataFrame:\n", team1_df.head())
        print("Team 2 DataFrame:\n", team2_df.head())
        return None  # Exit the function early if the column is missing

    # Fill NaN values in 'Points' column with 0.0
    team1_df['Points'] = pd.to_numeric(team1_df['Points'], errors='coerce').fillna(0.0)
    team2_df['Points'] = pd.to_numeric(team2_df['Points'], errors='coerce').fillna(0.0)

    # Find the optimal lineup (top 11 players) for each team
    team1_df = find_optimal_lineup(team1_df)
    team2_df = find_optimal_lineup(team2_df)

    # Define the position order for sorting
    position_order = ['G', 'D', 'M', 'F']
    for df in [team1_df, team2_df]:
        df['Position'] = pd.Categorical(df['Position'], categories=position_order, ordered=True)
        df.sort_values(by=['Position', 'Points'], ascending=[True, False], inplace=True)

    # Select the final columns to use
    team1_df = team1_df[['Player', 'Team', 'Position', 'Matchup', 'Points', 'Pos Rank']]
    team2_df = team2_df[['Player', 'Team', 'Position', 'Matchup', 'Points', 'Pos Rank']]

    # Format team DataFrames to use player names as the index
    team1_df.set_index('Player', inplace=True)
    team2_df.set_index('Player', inplace=True)

    # Return the final DataFrames and team names
    return team1_df, team2_df, team1_name, team2_name

In [56]:
def get_luck_adjusted_league_standings(draft_league_id):
    """
    Calculates the luck-adjusted league standings based on gameweek performance and fixture results.

    Parameters:
    - fixtures_df: A DataFrame with the following columns:
        - 'Team 1', 'Team 1 Score', 'Team 1 Result', 'Team 2', 'Team 2 Score', 'Team 2 Result'

    Returns:
    - standings_df: A DataFrame showing the adjusted standings based on luck and average weekly rank.
    """
    # Draft League API URLs
    league_url = f"https://draft.premierleague.com/api/league/{draft_league_id}/details"

    # Get league details, fixtures, and team details
    league_response = requests.get(league_url).json()

    # Extract the standings and league_entries sections of the JSON
    fixtures = league_response['matches']
    league_entries = league_response['league_entries']
    # Create a dictionary mapping entry_ids to team names
    team_names = {entry['id']: entry['entry_name'] for entry in league_entries}

    # Create a list to hold fixture details
    fixtures_list = []

    for fixture in fixtures:
        team1_id = fixture['league_entry_1']
        team2_id = fixture['league_entry_2']
        team1_name = team_names.get(team1_id)
        team2_name = team_names.get(team2_id)
        team1_score = fixture['league_entry_1_points']
        team2_score = fixture['league_entry_2_points']

        if (team1_score != 0) & (team2_score != 0):
            # Determine match results based on scores
            if team1_score > team2_score:
                team1_result = 'W'
                team2_result = 'L'
            elif team1_score < team2_score:
                team1_result = 'L'
                team2_result = 'W'
            else:
                team1_result = team2_result = 'D'

            fixtures_list.append({
                "Team 1": team1_name,
                "Team 1 Score": team1_score,
                "Team 1 Result": team1_result,
                "Team 2": team2_name,
                "Team 2 Score": team2_score,
                "Team 2 Result": team2_result,
            })

    # Convert fixtures_list to dataframe
    fixtures_df = pd.DataFrame(fixtures_list)
    
    # Step 1: Calculate the Week_Rank for each team based on scores for each gameweek
    fixtures_df['Week'] = fixtures_df.index // (len(fixtures_df) // len(fixtures_df['Team 1'].unique())) + 1

    # Create a long-format dataframe with each team's score and result for the week
    long_format_df = pd.concat([
        fixtures_df[['Team 1', 'Team 1 Score', 'Team 1 Result', 'Week']].rename(columns={'Team 1': 'Team', 'Team 1 Score': 'Score', 'Team 1 Result': 'Result'}),
        fixtures_df[['Team 2', 'Team 2 Score', 'Team 2 Result', 'Week']].rename(columns={'Team 2': 'Team', 'Team 2 Score': 'Score', 'Team 2 Result': 'Result'})
    ])

    # Step 2: Calculate the rank for each team in each week based on their score
    long_format_df['Week_Rank'] = long_format_df.groupby('Week')['Score'].rank(ascending=False, method='min')

    # Step 3: Calculate the actual results for each team based on their results (W, L, D)
    long_format_df['Points'] = long_format_df['Result'].apply(lambda x: 3 if x == 'W' else (1 if x == 'D' else 0))

    # Step 4: Calculate the average weekly rank (Fair_Rank) and total points (actual standings) for each team
    team_stats_df = long_format_df.groupby('Team').agg(
        Avg_GW_Rank=('Week_Rank', 'mean'),  # The "fair" rank based on average weekly score rank
        Total_Points=('Points', 'sum'),       # The actual points based on match results
        Avg_Score=('Score', 'mean')           # The average score across all weeks
    ).reset_index()

    # Step 5: Create the adjusted standings by ranking teams based on Avg_Week_Rank
    team_stats_df['Fair_Rank'] = team_stats_df['Avg_GW_Rank'].rank(ascending=True, method='min').astype(int)

    # Rank the teams based on their actual points (Total_Points)
    team_stats_df['Actual_Rank'] = team_stats_df['Total_Points'].rank(ascending=False, method='min')

    # Step 6: Add luck index: Difference between actual rank and fair rank
    team_stats_df['Luck_Index'] = team_stats_df['Fair_Rank'] - team_stats_df['Actual_Rank']

    # Step 7: Sort by Fair_Rank to get the adjusted standings
    team_stats_df = team_stats_df.sort_values(by='Fair_Rank')
    
    # Step 8: Format number values
    team_stats_df['Avg_GW_Rank'] = round(team_stats_df['Avg_GW_Rank'], 2)
    team_stats_df['Avg_Score'] = round(team_stats_df['Avg_Score'], 2)

    return team_stats_df

In [57]:
def get_draft_picks(league_id):
    """
    Fetches the draft picks for each team in the league and returns a DataFrame
    with pick order, player name, and team name.

    Parameters:
    - league_id: The ID of the FPL Draft league.

    Returns:
    - DataFrame containing draft picks with columns: pick, player_name, and team_name.
    """
    # Endpoints for draft picks and league details
    draft_url = f"https://draft.premierleague.com/api/draft/{league_id}/choices"
    league_details_url = f"https://draft.premierleague.com/api/league/{league_id}/details"

    # Fetch data
    draft_data = requests.get(draft_url).json()
    league_details = requests.get(league_details_url).json()

    # Fetch the player mapping
    player_mapping = get_fpl_player_mapping()  # Format: {player_id: {'Player': 'Name', ...}}

    # Create a mapping of entry ID to team name
    team_names = {entry['id']: entry['entry_name'] for entry in league_details['league_entries']}

    # Initialize a list to store draft picks
    draft_picks_list = []

    # Populate draft picks
    for choice in draft_data['choices']:
        pick_number = choice['index']
        team_id = choice['entry']  # Team ID
        team_name = choice['entry_name']  # Team name
        player_id = choice['element']  # Player ID

        # Get the player details from the mapping
        player_info = player_mapping.get(player_id, {'Player': f"Unknown ({player_id})"})
        player_name = player_info['Player']

        # Append the draft pick details to the list
        draft_picks_list.append({
            'pick': pick_number,
            'player_name': player_name,
            'team_name': team_name
        })

    # Convert the list of draft picks to a DataFrame
    draft_picks_df = pd.DataFrame(draft_picks_list, columns=['pick', 'player_name', 'team_name'])

    return draft_picks_df

In [58]:
def get_starting_team_composition(league_id):
    """
    Fetches the draft picks for each team in the league and returns a dictionary with team_id as the primary key 
    and the team_name field plus a player field with all the player names.

    Parameters:
    - league_id: The ID of the FPL Draft league.

    Returns:
    - draft_picks: A dictionary where keys are team IDs, and values are dictionaries with team name and player names.
    """
    # Endpoints for draft picks and player data
    draft_url = f"https://draft.premierleague.com/api/draft/{league_id}/choices"
    league_details_url = f"https://draft.premierleague.com/api/league/{league_id}/details"
    player_data_url = "https://draft.premierleague.com/api/bootstrap-static"

    # Fetch data
    draft_data = requests.get(draft_url).json()
    league_details = requests.get(league_details_url).json()
    player_data = requests.get(player_data_url).json()

    # Create a mapping of entry ID to team name
    team_names = {entry['id']: entry['entry_name'] for entry in league_details['league_entries']}

    # Create a mapping of player ID to player name
    player_mapping = {
        player['id']: f"{player['first_name']} {player['second_name']}"
        for player in player_data['elements']
    }

    # Initialize the draft picks dictionary
    draft_picks = {}

    # Populate draft picks
    for choice in draft_data['choices']:
        team_id = choice['entry']         # Team ID
        team_name = choice['entry_name']  # Team name
        player_id = choice['element']     # Player ID

        # Ensure the team_id key exists in the dictionary
        if team_id not in draft_picks:
            draft_picks[team_id] = {
                'team_name': team_name,
                'players': []  # Initialize player list
            }

        # Add the player name to the list
        player_name = player_mapping.get(player_id, f"Unknown ({player_id})")
        draft_picks[team_id]['players'].append(player_name)

    return draft_picks

In [59]:
def get_gameweek_fixtures(league_id, gameweek):
    # Draft League API URLs
    league_url = f"https://draft.premierleague.com/api/league/{league_id}/details"

    # Get league details, fixtures, and team details
    league_response = requests.get(league_url).json()

    # Extract the standings and league_entries sections of the JSON
    fixtures = league_response['matches']
    league_entries = league_response['league_entries']

    # Create a dictionary mapping entry_ids to team names and managers
    team_info = {entry['id']: (entry['entry_name'], entry['player_first_name'] + ' ' + entry['player_last_name'])
                 for entry in league_entries}

    # Create an empty list to add gameweek fixtures to
    gameweek_fixtures = []

    # Iterate over the fixtures, filter for the current_gameweek, and then format and add the fixture to list
    for fixture in fixtures:
        if fixture['event'] == gameweek:
            team1_id = fixture['league_entry_1']
            team2_id = fixture['league_entry_2']
            team1_name, team1_manager = team_info.get(team1_id, ("Unknown Team", "Unknown Manager"))
            team2_name, team2_manager = team_info.get(team2_id, ("Unknown Team", "Unknown Manager"))
            gameweek_fixtures.append(f"{team1_name} ({team1_manager}) vs {team2_name} ({team2_manager})")

    return gameweek_fixtures

In [60]:
def get_league_entries(league_id):
    """
    Fetches the league entries and creates a mapping of entry IDs to team names.

    Parameters:
    - league_id: The ID of the league.

    Returns:
    - A dictionary where keys are entry IDs, and values are team names.
    """
    url = f"https://draft.premierleague.com/api/league/{league_id}/details"
    response = requests.get(url).json()

    return {entry['entry_id']: entry['entry_name'] for entry in response['league_entries']}

In [61]:
def get_league_player_ownership(league_id):
    """
    Fetches the player ownership details for a given league and outputs a dictionary 
    where the keys are team IDs and the values include the team name and owned players grouped by position.

    Parameters:
    - league_id (int): The ID of the FPL Draft league.

    Returns:
    - Dictionary of player ownership in the format:
      {team_id: {'team_name': str, 'players': {'G': [...], 'D': [...], 'M': [...], 'F': [...]}}}
    """
    # URLs for API endpoints
    element_status_url = f"https://draft.premierleague.com/api/league/{league_id}/element-status"

    # Fetch player ownership data and league entries
    element_status = requests.get(element_status_url).json()['element_status']
    team_map = get_league_entries(league_id)  # Map of owner IDs to team names
    player_mapping = get_player_mapping()  # Map of player IDs to details (name, position, etc.)

    # Initialize the result dictionary
    league_player_ownership = {}

    # Populate the ownership details
    for status in element_status:
        player_id = status['element']
        owner_id = status['owner']

        # Skip players with no owner
        if owner_id is None:
            continue

        # Get player details
        player_info = player_mapping.get(player_id, {'Player': f"Unknown ({player_id})", 'Position': 'Unknown'})
        position = player_info['Position']
        player_name = player_info['Player']

        # Get team name from team_map
        team_name = team_map.get(owner_id, "Unknown Team")

        # Ensure the owner_id key exists in the dictionary
        if owner_id not in league_player_ownership:
            league_player_ownership[owner_id] = {
                'team_name': team_name,
                'players': {'G': [], 'D': [], 'M': [], 'F': []}  # Initialize player lists by position
            }

        # Append the player to the appropriate position list
        league_player_ownership[owner_id]['players'][position].append(player_name)

    return league_player_ownership

In [62]:
def get_league_player_dict_for_gameweek(league_id, gameweek):
    """
    Fetches the team compositions for all teams in the specified FPL draft league for a given gameweek.

    Parameters:
    - league_id (int): The ID of the FPL draft league.
    - gameweek (int): The gameweek for which to retrieve team compositions.

    Returns:
    - league_dict (dict): A dictionary where keys are team names, and values are dictionaries of player positions and players.
    """
    # Fetch league details to retrieve team names
    league_url = f"https://draft.premierleague.com/api/league/{league_id}/details"
    league_response = requests.get(league_url).json()

    # Create a dictionary to map entry IDs to team names
    team_name_map = {
        entry['entry_id']: entry['entry_name'] for entry in league_response['league_entries']
    }

    # Fetch all player data from the FPL API
    fpl_players_url = "https://draft.premierleague.com/api/bootstrap-static"
    fpl_data = requests.get(fpl_players_url).json()

    # Create a DataFrame with player information
    players_df = pd.DataFrame(fpl_data['elements'])
    players_df['Player'] = players_df['first_name'] + " " + players_df['second_name']
    players_df['Position'] = players_df['element_type'].map({1: 'G', 2: 'D', 3: 'M', 4: 'F'})

    # Map player IDs to their names and positions
    player_info = players_df.set_index('id')[['Player', 'Position']].to_dict(orient='index')

    # Fetch waiver transactions up to the given gameweek
    transactions = get_waiver_transactions_up_to_gameweek(league_id, gameweek)

    # Initialize the league dictionary with team compositions
    league_dict = {
        team_name: {'G': [], 'D': [], 'M': [], 'F': []} for team_name in team_name_map.values()
    }

    # Process draft picks to populate the initial team compositions
    draft_picks = get_starting_team_composition(league_id)
    for team_name, player_ids in draft_picks.items():
        for player_id in player_ids:
            player_data = player_info.get(player_id)
            if player_data:
                position = player_data['Position']
                player_name = player_data['Player']
                league_dict[team_name][position].append(player_name)

    # Apply transactions to update team compositions
    for tx in transactions:
        if tx['result'] == 'a':  # Only apply approved transactions
            team_id = tx['entry']
            team_name = team_name_map.get(team_id)

            if team_name:
                player_in = player_info.get(tx['element_in'])
                player_out = player_info.get(tx['element_out'])

                if player_out:
                    position_out = player_out['Position']
                    if player_out['Player'] in league_dict[team_name][position_out]:
                        league_dict[team_name][position_out].remove(player_out['Player'])

                if player_in:
                    position_in = player_in['Position']
                    league_dict[team_name][position_in].append(player_in['Player'])

    return league_dict

In [83]:
def get_fpl_player_mapping():
    """
    Fetches FPL player data from the FPL Draft API and returns it as a dictionary to link player ids to player names.

    Returns:
    - fpl_player_data: DataFrame with columns 'Player_ID', 'Player', 'Team', and 'Position'.
    """
    # Fetch data from the FPL Draft API
    player_url = "https://draft.premierleague.com/api/bootstrap-static"
    response = requests.get(player_url)

    # Extract relevant player information
    player_data = response.json()
    players = player_data['elements']
    teams = player_data.get('teams', [])

    # Create a mapping of player IDs to player information
    fpl_player_map = {}

    for player in players:
        player_id = player.get('id')
        first_name = player.get('first_name', '')
        second_name = player.get('second_name', '')
        full_name = f"{first_name} {second_name}"

        web_name = player.get('web_name', '').strip()
        if not web_name or web_name == full_name:
            web_name = None  # treat as missing if it's the same as full name or blank

        team_index = player.get('team', 0) - 1  # team index is 1-based
        position_index = player.get('element_type', 1) - 1

        team_short_name = teams[team_index]['short_name'] if 0 <= team_index < len(teams) else 'Unknown'
        position = ['G', 'D', 'M', 'F'][position_index] if 0 <= position_index < 4 else 'Unknown'

        fpl_player_map[player_id] = {
            'Player': full_name,
            'Web_Name': web_name,
            'Team': team_short_name,
            'Position': position
        }

    return fpl_player_map

In [64]:
def get_team_composition_for_gameweek(league_id, team_id, gameweek):
    """
    Determines the composition of a given FPL team for a specified gameweek.

    Parameters:
    - league_id (int): The ID of the league.
    - team_id (int): The team ID of the team to fetch.
    - gameweek: The gameweek for which to determine the team's composition.

    Returns:
    - DataFrame containing player name, team, and position for the specified gameweek.
    """
    # Fetch player and team mappings
    player_map = get_fpl_player_mapping()
    team_map = get_league_entries(league_id)

    # Initialize the team composition with the initial draft picks
    draft_picks = get_starting_team_composition(league_id)
    team_composition = set(draft_picks.get(team_id, {}).get('players', []))

    # Apply relevant transactions to the team composition
    transactions = get_waiver_transactions_up_to_gameweek(league_id, gameweek)
    for tx in transactions:
        if tx['entry'] == team_id and tx['result'] == 'a':
            # Convert player IDs to names using player_map
            player_in = player_map.get(tx['element_in'], {}).get('Player', f"Unknown ({tx['element_in']})")
            player_out = player_map.get(tx['element_out'], {}).get('Player', f"Unknown ({tx['element_out']})")

            # Update the team composition
            team_composition.remove(player_out)
            team_composition.add(player_in)

    # Convert the team composition to a DataFrame with player details
    player_data = [
        player_map.get(player_id, {'Player': player_name, 'Team': 'Unknown', 'Position': 'Unknown'})
        for player_name, player_id in [(player, next((k for k, v in player_map.items() if v['Player'] == player), None))
                                       for player in team_composition]
    ]

    fpl_players_df = pd.DataFrame(player_data)
    return(fpl_players_df)

In [146]:
def get_fpl_draft_league_standings(draft_league_id, show_luck_adjusted_standings=False):
    """
    Fetches and displays the current league standings for an FPL Draft league, with an option to show advanced statistics.

    Parameters:
    - draft_league_id (int or str): The ID of the FPL Draft league for which to fetch standings data.
    - show_luck_adjusted_standings (bool): If True, the function will show luck adjusted standings.

    Workflow:
    1. The function retrieves league details (standings and league entries) from the FPL Draft API.
    2. It extracts the standings and team/player information into separate DataFrames.
    3. The two DataFrames are merged on the team’s league entry ID to combine the standings with player information.
    4. The function renames and reformats the DataFrame, calculating the points based on wins (3 points per win) and draws (1 point per draw).
    5. It ranks teams based on their points, with higher points resulting in a better rank.
    6. Optionally, if `show_advanced_stats` is set to True, it computes advanced statistics such as the luck-adjusted standings 
       and merges them with the basic standings.

    Returns:
    - DataFrame: A pandas DataFrame containing the league standings, including:
        - Rank: The team's rank based on points.
        - Team: The name of the team.
        - Player: The full name of the player managing the team.
        - W: The number of matches won.
        - D: The number of matches drawn.
        - L: The number of matches lost.
        - PF: The total points scored by the team (points for).
        - PA: The total points scored against the team (points against).
        - Pts: The calculated points from wins and draws.
        - Optionally, if `show_advanced_stats` is True:
            - Avg_Week_Rank: The average rank based on weekly performance.
            - Avg_Score: The average score of the team across all gameweeks.
            - Fair_Rank: The team's fair rank based on weekly performance (luck-adjusted standings).
            - Luck_Index: The difference between the actual rank and the fair rank.

    Example Usage:
    standings_df = get_league_standings(49249, show_advanced_stats=True)  # Fetch league standings for league 49249 with advanced stats
    print(standings_df)
    """
    # Draft League API URLs
    league_url = f"https://draft.premierleague.com/api/league/{draft_league_id}/details"

    # Get league details, fixtures, and team details
    league_response = requests.get(league_url).json()

    # Extract the standings and league_entries sections of the JSON
    standings = league_response['standings']
    league_entries = league_response['league_entries']

    # Create DataFrame for standings
    standings_df = pd.DataFrame(standings)

    # Create DataFrame for league entries
    entries_df = pd.DataFrame(league_entries)

    # Merge standings with league entries on 'league_entry' and 'entry_id'
    league_standings_df = standings_df.merge(entries_df, left_on='league_entry', right_on='id', how='left')

    # Select and rename the required columns
    league_standings_df = league_standings_df[['entry_name', 'player_first_name', 'player_last_name', 'matches_won',
                                               'matches_drawn', 'matches_lost', 'points_for', 'points_against']]
    league_standings_df.columns = ['Team', 'Player Name', 'Player Last Name', 'W', 'D', 'L', 'PF', 'PA']

    # Concatenating 'Player Name' and 'Player Last Name' using .loc[] to avoid the warning
    league_standings_df.loc[:, 'Player'] = league_standings_df['Player Name'] + ' ' + league_standings_df['Player Last Name']

    # Drop the 'Player Last Name' column if no longer needed
    league_standings_df = league_standings_df.drop(columns=['Player Last Name'])

    # Calculate the 'Pts' based on wins and draws
    league_standings_df['Pts'] = league_standings_df['W'] * 3 + league_standings_df['D'] * 1

    # Convert 'Total Points' to numeric for ranking
    league_standings_df['PF'] = pd.to_numeric(league_standings_df['PF'], errors='coerce')

    # Add a Rank column based on 'Total Points'
    league_standings_df['Rank'] = league_standings_df['Pts'].rank(ascending=False, method='min').astype(int)

    # Sort by Rank
    league_standings_df = league_standings_df.sort_values('Rank')

    # Reorder the columns
    league_standings_df = league_standings_df[['Rank', 'Team', 'Player', 'W', 'D', 'L', 'PF', 'PA', 'Pts']]
    
    # If show_advanced_stats is true, add the avg_gw_rank and avg_gw_score to the dataframe
    if show_luck_adjusted_standings == True:
        # Compute the luck adjusted standings
        luck_adjusted_df = get_luck_adjusted_league_standings(draft_league_id)
        # Return the results
        return(luck_adjusted_df)
    else:
        # Set the index
        league_standings_df.set_index('Rank', inplace=True)
        # Return the results
        return(league_standings_df)

In [66]:
def get_team_id_by_name(league_id, team_name):
    """
    Converts a team name to its corresponding team ID in the specified FPL league.

    Parameters:
    - league_id (int): The ID of the FPL league.
    - team_name (str): The name of the team to search for.

    Returns:
    - team_id (int): The ID of the team with the given name.

    Raises:
    - ValueError: If the team name is not found in the league.
    """
    # Fetch league entries to map team names to IDs
    team_map = dict(get_league_entries(league_id))  # Ensure team_map is a dictionary

    # Normalize the input team name
    normalized_team_name = normalize_apostrophes(team_name)

    # Search for the team ID by normalized team name
    team_id = next((id for id, name in team_map.items() if normalize_apostrophes(name) == normalized_team_name), None)

    if team_id is None:
        raise ValueError(f"Team '{team_name}' not found in the league.")

    return team_id

In [67]:
def get_waiver_transactions_up_to_gameweek(league_id, gameweek):
    """
    Fetches all transactions (waivers, free agent moves, etc.) up to the selected gameweek.

    Parameters:
    - league_id: The league ID for transactions.
    - gameweek: The gameweek to fetch transactions up to.

    Returns:
    - transactions: A list of transactions that occurred up to the given gameweek.
    """
    transaction_url = f"https://draft.premierleague.com/api/draft/league/{league_id}/transactions"
    transaction_response = requests.get(transaction_url)
    transactions = transaction_response.json()['transactions']

    # Filter transactions by gameweek
    filtered_transactions = [tx for tx in transactions if tx['event'] <= gameweek]

    return filtered_transactions

In [68]:
def show_fixture_results(draft_league_id, gameweek):
    """
    Displays the results of the fixtures for a specific gameweek in an FPL Draft league.

    Parameters:
    - draft_league_id (int or str): The ID of the FPL Draft league to fetch fixture data from.
    - gameweek (int): The specific gameweek for which to display fixture results.

    Workflow:
    1. The function retrieves league details and fixtures from the FPL Draft API.
    2. A dictionary is created to map `entry_id` to the corresponding team name.
    3. The function iterates over the fixtures and filters out only those that match the specified gameweek.
    4. For each fixture, the results (Win, Loss, Draw) are determined by comparing the scores of both teams.
    5. If both teams have non-zero scores, the fixture details (team names, scores, results) are appended to a list.
    6. Finally, the fixture results are printed as a pandas DataFrame. If no results are found, an appropriate message is displayed.

    Example Usage:
    show_fixture_results(49249, 6)  # Fetch and display the results for league 49249, gameweek 6.
    """
    # Draft League API URLs
    league_url = f"https://draft.premierleague.com/api/league/{draft_league_id}/details"

    # Get league details, fixtures, and team details
    league_response = requests.get(league_url).json()

    # Extract the standings and league_entries sections of the JSON
    fixtures = league_response['matches']
    league_entries = league_response['league_entries']
    # Create a dictionary mapping entry_ids to team names
    team_names = {entry['id']: entry['entry_name'] for entry in league_entries}

    # Create a list to hold fixture details
    fixtures_list = []

    for fixture in fixtures:
        # Filter to just the gameweek of interest
        if fixture['event'] == gameweek:
            team1_id = fixture['league_entry_1']
            team2_id = fixture['league_entry_2']
            team1_name = team_names.get(team1_id)
            team2_name = team_names.get(team2_id)
            team1_score = fixture['league_entry_1_points']
            team2_score = fixture['league_entry_2_points']

            if (team1_score != 0) & (team2_score != 0):
                # Determine match results based on scores
                if team1_score > team2_score:
                    team1_result = 'W'
                    team2_result = 'L'
                elif team1_score < team2_score:
                    team1_result = 'L'
                    team2_result = 'W'
                else:
                    team1_result = team2_result = 'D'

                fixtures_list.append({
                    "Team 1": team1_name,
                    "Team 1 Score": team1_score,
                    "Team 1 Result": team1_result,
                    "Team 2": team2_name,
                    "Team 2 Score": team2_score,
                    "Team 2 Result": team2_result,
                })

    # Convert the list into a DataFrame
    if fixtures_list:
        fixtures_df = pd.DataFrame(fixtures_list)
        print(f"Results for Gameweek {gameweek}:")
        print(fixtures_df)
    else:
        print(f"No results available for Gameweek {gameweek}.")

In [69]:
def find_top_waivers(player_projections, team_dict, limit=None, fdr_weight=1.0, gameweeks_ahead=3):
    """
    Analyze and display waiver wire options by factoring in projected points and fixture difficulty.

    Parameters:
    - player_projections: DataFrame containing player projections.
    - team_dict: Dictionary containing the league's team rosters.
    - limit: Number of top players to return.
    - fdr_weight: Weight of fixture difficulty in calculating priority score.
    - gameweeks_ahead: Number of upcoming gameweeks to consider for FDR.

    Returns:
    - DataFrame of top waiver options.
    """
    # Step 1: Fetch FDR data and map team names to IDs
    fdr_data = get_fixture_difficulty()
    fpl_player_df = get_fpl_player_data()
    
    # Step 2: Flatten the team_dict to get all taken players
    all_taken_players = []
    for team_data in team_dict.values():
        if isinstance(team_data, dict):
            for position_players in team_data.values():
                all_taken_players.extend(position_players)

    # Step 3: Create a DataFrame of the taken players
    taken_players_df = pd.DataFrame({'Player': all_taken_players})
    
    # Merge taken players with player data
    taken_players_df = pd.merge(
        taken_players_df, fpl_player_df, on='Player', how='left'
    ).set_index('Player_ID')

    # Step 4: Merge taken players with projections to resolve name discrepancies
    print(taken_players_df)
    print(player_projections)
    taken_players_projections = merge_fpl_players_and_projections(taken_players_df, player_projections)

    # Step 5: Find available players (not on any team)
    available_players = player_projections[
        ~player_projections['Player'].isin(taken_players_projections['Player'])
    ]

    # Step 6: Calculate FDR for each player
    available_players['FDR'] = available_players.apply(
        lambda row: calculate_average_fdr(row['Team'], row['Position'], fdr_data, gameweeks_ahead),
        axis=1
    )

    # Step 7: Calculate Priority Score
    available_players['Priority Score'] = available_players['Points'] - (fdr_weight * available_players['FDR'])

    # Step 8: Sort by Priority Score and limit results
    available_players = available_players.sort_values(by='Priority Score', ascending=False)

    if limit:
        available_players = available_players.head(limit)

    # Step 9: Return relevant columns
    return available_players[['Player', 'Team', 'Position', 'Points', 'FDR', 'Priority Score']]


In [70]:
def merge_fpl_players_and_projections(fpl_players_df, projections_df, fuzzy_threshold=80, lower_fuzzy_threshold=60):
    """
    Merges the FPL player data with Rotowire projections based on player name, team, and position.

    Parameters:
    - fpl_players_df: DataFrame with FPL players ['player', 'team', 'position'].
    - projections_df: DataFrame with Rotowire projections.
    - fuzzy_threshold: Default fuzzy matching threshold for player names.
    - lower_fuzzy_threshold: Lower threshold if team and position match.

    Returns:
    - merged_df: DataFrame with players, projections, and any missing players shown with NA values.
    """

    def find_best_match(fpl_player, fpl_team, fpl_position, candidates):
        """Finds the best match for a player using fuzzy matching."""
        match, score = process.extractOne(fpl_player, candidates)

        matched_row = projections_df[projections_df['Player'] == match]
        if not matched_row.empty:
            match_team = matched_row.iloc[0]['Team']
            match_position = matched_row.iloc[0]['Position']

            if match_team == fpl_team and match_position == fpl_position and score >= lower_fuzzy_threshold:
                return match

        if score >= fuzzy_threshold:
            return match

        return None

    merged_data = []

    for _, fpl_row in fpl_players_df.iterrows():
        fpl_player = fpl_row['Player']
        fpl_team = fpl_row['Team']
        fpl_position = fpl_row['Position']

        candidates = projections_df['Player'].tolist()
        best_match = find_best_match(fpl_player, fpl_team, fpl_position, candidates)

        if best_match:
            matched_row = projections_df[projections_df['Player'] == best_match].iloc[0]
            merged_data.append({
                'Player': matched_row['Player'],
                'Team': matched_row['Team'],
                'Matchup': matched_row.get('Matchup', ''),
                'Position': matched_row['Position'],
                'Price': matched_row.get('Price', float('nan')),
                'TSB%': matched_row.get('tsb', float('nan')),
                'Points': matched_row['Points'],
                'Pos Rank': matched_row.get('Pos Rank', 'NA')
            })
        else:
            merged_data.append({
                'Player': fpl_player,
                'Team': fpl_team,
                'Matchup': 'N/A',
                'Position': fpl_position,
                'Price': float('nan'),
                'TSB%': float('nan'),
                'Points': float('nan'),
                'Pos Rank': 'NA'
            })

    # Create the DataFrame
    merged_df = pd.DataFrame(merged_data)

    # Reorder columns
    merged_df = merged_df[['Player', 'Team', 'Matchup', 'Position', 'Price', 'TSB%', 'Points', 'Pos Rank']]

    # Ensure 'Pos Rank' is integer or 'NA'
    merged_df['Pos Rank'] = pd.to_numeric(merged_df['Pos Rank'], errors='coerce').fillna(-1).astype(int)
    merged_df['Pos Rank'] = merged_df['Pos Rank'].replace(-1, 'NA')

    # Set index to represent overall rank, starting at 1
    merged_df.index = pd.RangeIndex(start=1, stop=len(merged_df) + 1, step=1)

    return merged_df

In [71]:
def normalize_apostrophes(text):
    """
    Normalizes text by converting different apostrophe types to a standard straight apostrophe.

    Parameters:
    - text (str): The text to normalize.

    Returns:
    - str: The normalized text.
    """
    if text is None:
        return None
    # Normalize Unicode and replace curly apostrophes with straight apostrophes
    return unicodedata.normalize('NFKC', text).replace('’', "'").strip().lower()


In [72]:
def plot_team_points_over_time(draft_league_id):
    """
    Fetches the gameweek results for all teams in an FPL Draft league and plots the total points over time for each team.
    Only includes matches that have been played (up to the current gameweek).

    Parameters:
    - draft_league_id (int or str): The ID of the FPL Draft league to retrieve data from.

    Returns:
    - Plotly figure displaying the total points by team over time (gameweek).
    """
    # Step 1: Get the current gameweek
    current_gameweek = get_current_gameweek()

    # Step 2: Fetch league details
    league_url = f"https://draft.premierleague.com/api/league/{draft_league_id}/details"
    league_response = requests.get(league_url).json()

    # Step 3: Create a dictionary mapping entry_ids to team names
    league_entries = league_response['league_entries']
    team_names = {entry['id']: entry['entry_name'] for entry in league_entries}

    # Step 4: Extract matches data from the response
    matches = league_response['matches']

    # Step 5: Create a DataFrame to track points for each team over time
    total_points = {team_id: 0 for team_id in team_names.keys()}  # Dictionary to hold cumulative points
    data = []

    # Step 6: Process each match and accumulate points by gameweek, ignoring future matches
    for match in matches:
        gameweek = match['event']
        if gameweek >= current_gameweek:  # Ignore future matches
            continue

        team1_id = match['league_entry_1']
        team2_id = match['league_entry_2']
        team1_points = match['league_entry_1_points']
        team2_points = match['league_entry_2_points']
        
        # Update cumulative points for both teams
        total_points[team1_id] += team1_points
        total_points[team2_id] += team2_points
        
        # Append points data to DataFrame for plotting
        data.append({'Team': team_names[team1_id], 'Gameweek': gameweek, 'Total Points': total_points[team1_id]})
        data.append({'Team': team_names[team2_id], 'Gameweek': gameweek, 'Total Points': total_points[team2_id]})

    # Step 7: Create a DataFrame from the data
    df = pd.DataFrame(data)

    # Step 8: Plot the data using Plotly
    fig = px.line(df, x="Gameweek", y="Total Points", color="Team",
                  labels={"Total Points": "Total Points", "Gameweek": "Gameweek", "Team": "Team"},
                  title="Team Total Points Over Time (Gameweek)")
    
    # Customize the layout
    fig.update_layout(xaxis_title="Gameweek", yaxis_title="Total Points", hovermode="x unified")
    
    # Show the figure
    fig.show()

In [73]:
def plot_league_points_over_time(draft_league_id):
    """
    Fetches the gameweek results for all teams in an FPL Draft league and plots the total league points
    over time based on wins, ties, and losses. Points are awarded: 3 for a win, 1 for a tie, and 0 for a loss.

    Parameters:
    - draft_league_id (int or str): The ID of the FPL Draft league to retrieve data from.

    Returns:
    - Plotly figure displaying the total league points by team over time (gameweek).
    """
    # Step 1: Get the current gameweek
    current_gameweek = get_current_gameweek()

    # Step 2: Fetch league details
    league_url = f"https://draft.premierleague.com/api/league/{draft_league_id}/details"
    league_response = requests.get(league_url).json()

    # Step 3: Create a dictionary mapping entry_ids to team names
    league_entries = league_response['league_entries']
    team_names = {entry['id']: entry['entry_name'] for entry in league_entries}

    # Step 4: Initialize league points for each team and data for plotting
    league_points = {team_id: 0 for team_id in team_names.keys()}  # Holds cumulative league points
    data = []  # To store data for plotting

    # Step 5: Process match results and accumulate points by gameweek
    matches = league_response['matches']
    for match in matches:
        gameweek = match['event']
        if gameweek > current_gameweek:  # Ignore future matches
            continue

        team1_id = match['league_entry_1']
        team2_id = match['league_entry_2']
        team1_points = match['league_entry_1_points']
        team2_points = match['league_entry_2_points']

        # Determine match result and update league points
        if team1_points > team2_points:
            team1_result = 'W'
            team2_result = 'L'
            league_points[team1_id] += 3  # 3 points for a win
        elif team1_points < team2_points:
            team1_result = 'L'
            team2_result = 'W'
            league_points[team2_id] += 3  # 3 points for a win
        else:
            team1_result = team2_result = 'T'
            league_points[team1_id] += 1  # 1 point for a tie
            league_points[team2_id] += 1

        # Step 6: Append data for both teams to track their points by gameweek
        data.append({'Team': team_names[team1_id], 'Gameweek': gameweek, 'Total League Points': league_points[team1_id]})
        data.append({'Team': team_names[team2_id], 'Gameweek': gameweek, 'Total League Points': league_points[team2_id]})

    # Step 7: Create a DataFrame from the data
    df = pd.DataFrame(data)

    # Step 8: Plot the data using Plotly
    fig = px.line(df, x="Gameweek", y="Total League Points", color="Team",
                  labels={"Total League Points": "Total League Points", "Gameweek": "Gameweek", "Team": "Team"},
                  title="Team Total League Points Over Time (Gameweek)")
    
    # Customize the layout
    fig.update_layout(xaxis_title="Gameweek", yaxis_title="Total League Points", hovermode="x unified")
    
    # Show the figure
    fig.show()

In [74]:
def scrape_rotowire_lineups(url="https://www.rotowire.com/soccer/lineups.php"):
    """
    Scrapes the Rotowire Soccer Lineups page to extract the projected lineups for all matchups,
    excluding players listed in the Injuries section.

    Parameters:
    - url (str): The URL of the Rotowire lineups page.

    Returns:
    - DataFrame containing the team names, player names, and positions.
    """
    # Send a request to the Rotowire lineups page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Initialize an empty list to store match data
    matchups = []

    # Find all lineup sections (home and away matchups)
    lineup_sections = soup.find_all('div', class_='lineup__main')

    # Iterate through each section to extract team and player data
    for section in lineup_sections:
        try:
            # Extract home and away team names
            home_team = section.find_previous('div', class_='lineup__mteam is-home').text.strip()
            away_team = section.find_previous('div', class_='lineup__mteam is-visit').text.strip()

            # Extract players while excluding those listed in the Injuries section
            home_players = extract_players(section, 'home', home_team)
            away_players = extract_players(section, 'visit', away_team)

            # Add players to the matchups list
            matchups.extend(home_players + away_players)

        except AttributeError as e:
            print(f"Error parsing section: {e}")

    # Convert the data to a pandas DataFrame
    lineups_df = pd.DataFrame(matchups, columns=['Team', 'Position', 'Player'])

    return lineups_df

In [75]:
def extract_players(section, team_type, team_name):
    """
    Extracts valid players from the given section, excluding those listed in the Injuries section.

    Parameters:
    - section (BeautifulSoup): The section containing the team’s lineup.
    - team_type (str): 'home' or 'visit' to determine the team type.
    - team_name (str): The name of the team.

    Returns:
    - list: A list of tuples containing (Team, Position, Player) for valid players.
    """
    player_list = []
    injuries_section_reached = False  # Track if the Injuries section has been reached

    # Find the correct lineup section for the team
    players_section = section.find('ul', class_=f'lineup__list is-{team_type}')

    if players_section:
        for item in players_section.find_all('li', class_='lineup__player'):
            # Check if we reached the Injuries section
            if item.find_previous_sibling('li', class_='lineup__title is-middle'):
                injuries_section_reached = True

            # Skip players if we are in the Injuries section
            if not injuries_section_reached:
                try:
                    position = item.find('div', class_='lineup__pos').text.strip()
                    player_name = item.find('a').text.strip()
                    player_list.append((team_name, position, player_name))
                except AttributeError:
                    continue  # Skip if position or player name is missing

    return player_list

In [76]:
def plot_soccer_field(player_df, team_name):
    """
    Plots players on a soccer field based on their positions for a specific team.

    Parameters:
    - player_df (pd.DataFrame): A DataFrame containing 'Position' and 'Player' columns.
    - team_name (str): The name of the team, displayed as the title above the field.
    """
    # Dictionary mapping teams to random colors
    team_colors = {
        "Tottenham Hotspur": "#%06x" % random.randint(0, 0xFFFFFF),
        "Arsenal": "#%06x" % random.randint(0, 0xFFFFFF),
        "Manchester City": "#%06x" % random.randint(0, 0xFFFFFF),
        "Chelsea": "#%06x" % random.randint(0, 0xFFFFFF),
        "Liverpool": "#%06x" % random.randint(0, 0xFFFFFF),
        "West Ham United": "#%06x" % random.randint(0, 0xFFFFFF),
        "Aston Villa": "#%06x" % random.randint(0, 0xFFFFFF),
        "Newcastle United": "#%06x" % random.randint(0, 0xFFFFFF),
        "Manchester United": "#%06x" % random.randint(0, 0xFFFFFF)
    }
    
    # Define positions with (x, y) coordinates
    position_mapping = {
        'GK': (5, 0),  # Goalkeeper
        'DL': (2, 1.5), 'DC': (5, 1), 'DR': (8, 1.5),  # Defenders
        'DML': (2, 2.5), 'DMC': (5, 2.5), 'DMR': (8, 2.5),  # Defensive Midfielders
        'ML': (1, 3.5), 'MC': (5, 3), 'MR': (9, 3.5),  # Midfielders
        'AML': (2, 3.5), 'AMC': (5, 3.5), 'AMR': (8, 3.5),  # Attacking Midfielders
        'FL': (2, 5), 'FWL': (3, 4.5), 'FC': (5, 5.5), 'FW': (5, 5), 'FWR': (7, 4.5), 'FR': (8, 5)  # Forwards
    }


    # Get the team color, default to black if not found
    team_color = team_colors.get(team_name, "#000000")

    # Group players by position
    grouped_players = defaultdict(list)
    for _, row in player_df.iterrows():
        grouped_players[row['Position']].append(row['Player'])

    # Create a Plotly figure
    fig = go.Figure()

    # Draw field boundary and half-line
    fig.add_shape(type="rect", x0=0, y0=-0.5, x1=10, y1=6.5, line=dict(color="green", width=2))
    fig.add_shape(type="line", x0=5, y0=-0.5, x1=5, y1=6.5, line=dict(color="white", width=1, dash="dash"))

    # Add players to the field
    for position, players in grouped_players.items():
        for i, player_name in enumerate(players):
            x, y = position_mapping.get(position, (5, 3))  # Default to MC if not found

            # Adjust for two players in the same central position
            if len(players) == 2 and position in ['DC', 'MC', 'FC', 'DMC', 'AMC']:
                x += -1 if i == 0 else 1  # Shift left and right

            # Adjust for three players in the same central position
            elif len(players) == 3 and position in ['DC', 'MC', 'FC', 'DMC', 'AMC']:
                if i == 0:
                    x -= 1  # Shift left
                elif i == 1:
                    y -= .5 # Shift down slightly
                elif i == 2:
                    x += 1  # Shift right

            # Spread out three-player lines (AML, AMC, AMR and DML, DMC, DMR)
            elif position in ['AML', 'AMC', 'AMR', 'DML', 'DMC', 'DMR']:
                if position.endswith('L'):
                    x -= 1
                elif position.endswith('R'):
                    x += 1

            # Add player marker with team-specific color and hover info
            fig.add_trace(go.Scatter(
                x=[x], y=[y], mode='markers+text',
                marker=dict(size=15, color=team_color),
                text=player_name,
                textposition="top center",
                hovertemplate=f"Name: {player_name}<br>Position: {position}<br>Coordinates: ({x}, {y})<extra></extra>"
            ))

    # Update layout to avoid player cutoff and set title
    fig.update_layout(
        title_text=f"{team_name} Lineup",
        showlegend=False,
        width=800, height=500,
        xaxis=dict(range=[0, 10], visible=False),
        yaxis=dict(range=[-1, 7], visible=False),  # Padded y-axis
        plot_bgcolor="green"
    )

    # Display the plot
    fig.show()

In [77]:
def position_converter(element_type):
    """Converts element type to position name."""
    return {1: 'G', 2: 'D', 3: 'M', 4: 'F'}.get(element_type, 'Unknown')

## Set Key Variables 

In [78]:
## Set needed ID variables
brandon_draft_league_id = 49249
brandon_draft_team_id = 189880

# Set dummy gameweek values
gameweek = 1
current_gameweek = get_current_gameweek()

# Draft League API URLs
draft_team_endpoint = f"https://draft.premierleague.com/api/entry/{brandon_draft_team_id}"
draft_league_endpoint = f'https://draft.premierleague.com/api/league/{brandon_draft_league_id}/details'
draft_fixtures_endpoint = f"https://draft.premierleague.com/api/league/{brandon_draft_league_id}/fixtures"
game_status_endpoint = f'https://draft.premierleague.com/api/game'
draft_gw_lineups_endpoint = f'https://draft.premierleague.com/api/entry/{brandon_draft_team_id}/event/{gameweek}'
waivers_endpoint = f'https://draft.premierleague.com/api/draft/league/{brandon_draft_league_id}/transactions'
draft_picks_endpoint = f'https://draft.premierleague.com/api/draft/{brandon_draft_league_id}/choices'

## Import Data

### Read in Rotowire Fantasy Premier League rankings data using BeautifulSoup

In [79]:
# Example usage
url = get_rotowire_rankings_url()
print("Rotowire Rankings URL: ", url)
fpl_player_projections = get_rotowire_player_projections(url)
fpl_player_projections.head(10)

Rotowire Rankings URL:  https://www.rotowire.com/soccer/article/fantasy-premier-league-player-rankings-gameweek-33-91994


,Overall Rank,Player,Team,Matchup,Position,Price,TSB %,Points,Pos Rank,Value
1,1,Bukayo Saka,ARS,ARS at IPS,M,10.5,21.9,10.39,1,0.989524
2,2,Mohamed Salah,LIV,LIV at LEI,M,13.8,71.6,10.33,2,0.748551
3,3,Marcus Rashford,AVL,AVL v. NEW,M,6.7,7.4,9.68,3,1.444776
4,4,Mikel Merino,ARS,ARS at IPS,M,6.1,3.1,9.15,4,1.500000
5,5,Gabriel Martinelli,ARS,ARS at IPS,M,6.5,2.2,8.96,5,1.378462
6,6,Omar Marmoush,MCI,MCI at EVE,F,7.6,30.2,8.71,1,1.146053
7,7,Eberechi Eze,CRY,CRY v. BOU,M,6.9,14.3,8.64,6,1.252174
8,8,Ismaila Sarr,CRY,CRY v. BOU,M,5.8,12.0,8.51,7,1.467241
9,9,Leandro Trossard,ARS,ARS at IPS,M,6.7,3.0,8.47,8,1.264179
10,10,Kevin De Bruyne,MCI,MCI at EVE,M,9.3,4.6,8.44,9,0.907527


### Pull FPL Draft Teams

In [80]:
# Example usage
draft_picks_df = get_draft_picks(brandon_draft_league_id)

KeyError: 'web_name'

In [ ]:
starting_teams = get_starting_team_composition(brandon_draft_league_id)
print(starting_teams)

In [ ]:
# Fetch player and team mappings
player_map = get_fpl_player_mapping(fpl_player_projections)
player_map

In [ ]:
fpl_player_projections

In [ ]:
# Fetch player and team mappings
player_map = get_fpl_player_mapping()
team_map = get_league_entries(brandon_draft_league_id)
team_name = 'Stoned Squirrels'
# Get the entry ID for the selected team name
entry_id = next((k for k, v in team_map.items() if v == team_name), None)
print("Entry ID: ", entry_id)
print("Team Name: ", team_name)
if entry_id is None:
    raise ValueError(f"Team '{team_name}' not found in the league.")

# Initialize the team composition with the initial draft picks
draft_picks = get_starting_team_composition(brandon_draft_league_id)
print("Draft Picks: \n", draft_picks, "\n")
team_composition = set(draft_picks.get(entry_id, {}).get('players', []))
print("Initial Team Composition: \n", team_composition, "\n")

# Apply relevant transactions to the team composition
transactions = get_waiver_transactions_up_to_gameweek(brandon_draft_league_id, get_current_gameweek())
for tx in transactions:
    if tx['entry'] == entry_id and tx['result'] == 'a':
        # Convert player IDs to names using player_map
        player_in = player_map.get(tx['element_in'], {}).get('Player', f"Unknown ({tx['element_in']})")
        player_out = player_map.get(tx['element_out'], {}).get('Player', f"Unknown ({tx['element_out']})")

        # Update the team composition
        team_composition.remove(player_out)
        print("Player Out: ", player_out)
        team_composition.add(player_in)
        print("Player In: ", player_in)

        
# Convert the team composition to a DataFrame with player details
player_data = [
    player_map.get(player_id, {'Player': player_name, 'Team': 'Unknown', 'Position': 'Unknown'})
    for player_name, player_id in [(player, next((k for k, v in player_map.items() if v['Player'] == player), None))
                                   for player in team_composition]
]

fpl_players_df = pd.DataFrame(player_data)
print("FPL Players : \n", fpl_players_df)

### Import FPL Player Stats from FPL Draft API 

In [ ]:
player_statistics = pull_fpl_player_stats()
player_statistics.head(10)

In [ ]:
player_statistics.columns

#### Expected vs. Actual Goal Involvements (scatter)

In [ ]:
# Replace NaN or negative values in total_points to avoid invalid sizes
player_statistics["total_points"] = player_statistics["total_points"].fillna(0)
player_statistics["total_points"] = player_statistics["total_points"].apply(lambda x: max(x, 1))  # Ensure positive size

# Drop rows where expected_goal_involvements is NaN after conversion
player_statistics = player_statistics.dropna(subset=["expected_goal_involvements"])

# Create hover text with bold labels
player_statistics["hover_text"] = (
    "<b>Player:</b> " + player_statistics["player"] +
    "<br><b>Position:</b> " + player_statistics["position_abbrv"] +
    "<br><b>Team Name:</b> " + player_statistics["team_name"] +
    "<br><b>Expected Goals:</b> " + player_statistics["expected_goal_involvements"].astype(str) +
    "<br><b>Actual Goal Involvement:</b> " + player_statistics["actual_goal_involvements"].astype(str) +
    "<br><b>Total Points:</b> " + player_statistics["total_points"].astype(str)
)

# Create scatter plot
fig = px.scatter(
    player_statistics,
    x="expected_goal_involvements",
    y="actual_goal_involvements",
    color="position_abbrv",
    hover_data={"hover_text": True},  # Use formatted hover text
    size="total_points",
    title="Expected vs. Actual Goal Involvements",
    labels={
        "expected_goal_involvements": "Expected Goal Involvements (xGI)",
        "actual_goal_involvements": "Actual Goal Involvements (Goals + Assists)",
    },
)

# Remove other labels and only display hover_text
fig.update_traces(hovertemplate="%{customdata[0]}", marker=dict(opacity=0.8))

# Update x-axis to ensure ascending order and consistent intervals
fig.update_layout(
    xaxis_title="Expected Goal Involvements (xGI)",
    yaxis_title="Actual Goal Involvements (Goals + Assists)",
    legend_title="Position",
    template="plotly_dark",
    xaxis=dict(
        tickmode="linear",  
        dtick=5,  # Adjust tick intervals for readability
        showgrid=True,
        zeroline=True,
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
    ),
)

# Show plot
fig.show()

#### Positional Point Distributions (box-plot) 

In [ ]:
# Create the box plot
fig = px.box(
    player_statistics,
    x='position_abbrv', 
    y='total_points',
    color='position_abbrv',  # Different colors for each position
    title="Distribution of Total Points by Position",
    labels={'position_abbrv': 'Position', 'total_points': 'Total Points'},
    hover_data=['player', 'team_name']
)

# Update layout for better readability
fig.update_layout(
    xaxis_title="Player Position",
    yaxis_title="Total Points",
    boxmode="group",  # Groups positions together
    template="plotly_white"
)

# Show the figure
fig.show()

#### Top Goal Scorers

In [ ]:
# Get top 10 goal scorers
top_scorers = player_statistics.nlargest(10, 'goals_scored')

# Create the horizontal bar plot
fig = px.bar(
    top_scorers,
    y='player',
    x='goals_scored',
    color='team_name',
    text='goals_scored',  # Display Goals at the end of the bars
    title="Top Goal Scorers",
    labels={'player': 'Player', 'goals_scored': 'Goals', 'team_name': 'Team'},
    orientation='h'  # Horizontal bar chart
)

# Update text and hover template
fig.update_traces(
    texttemplate='%{x}', 
    textposition='outside',
    hovertemplate='<b>Player</b> = %{y}<br>' +
                  '<b>Team</b> = %{customdata[0]}<br>' +
                  '<b>Goals</b> = %{x}<extra></extra>',
    customdata=top_scorers[['team_name']].values
)

# Reverse the order so the highest scorer is at the top
fig.update_layout(
    yaxis=dict(categoryorder='total ascending')  # Ensures correct descending order
)

# Display the figure
fig.show()

#### Top Assists

In [ ]:
# Get top 10 assists
top_assisters = player_statistics.nlargest(10, 'assists')

# Create the horizontal bar plot
fig = px.bar(
    top_assisters,
    y='player',
    x='assists',
    color='team_name',
    text='assists',  # Display Assists at the end of the bars
    title="Top Assisters",
    labels={'player': 'Player', 'assists': 'Assists', 'team_name': 'Team'},
    orientation='h'  # Horizontal bar chart
)

# Update text and hover template
fig.update_traces(
    texttemplate='%{x}', 
    textposition='outside',
    hovertemplate='<b>Player</b> = %{y}<br>' +
                  '<b>Team</b> = %{customdata[0]}<br>' +
                  '<b>Assists</b> = %{x}<extra></extra>',
    customdata=top_assisters[['team_name']].values
)

# Reverse the order so the highest scorer is at the top
fig.update_layout(
    yaxis=dict(categoryorder='total ascending')  # Ensures correct descending order
)

# Display the figure
fig.show()

#### Top Clean Sheets 

In [ ]:
# Filter only goalkeepers and defenders
clean_sheets_df = player_statistics[player_statistics['position_abbrv'].isin(['GK', 'DEF'])]

# Get top 10 clean sheets
top_clean_sheets = clean_sheets_df.nlargest(10, 'clean_sheets')

# Create the horizontal bar plot
fig = px.bar(
    top_clean_sheets,
    y='player',
    x='clean_sheets',
    color='team_name',
    text='clean_sheets',  # Display Clean Sheets at the end of the bars
    title="Top Clean Sheets",
    labels={'player': 'Player', 'clean_sheets': 'Clean Sheets', 'team_name': 'Team'},
    orientation='h'  # Horizontal bar chart
)

# Update text and hover template
fig.update_traces(
    texttemplate='%{x}', 
    textposition='outside',
    hovertemplate='<b>Player</b> = %{y}<br>' +
                  '<b>Team</b> = %{customdata[0]}<br>' +
                  '<b>Clean Sheets</b> = %{x}<extra></extra>',
    customdata=top_clean_sheets[['team_name']].values
)

# Reverse the order so the highest scorer is at the top
fig.update_layout(
    yaxis=dict(categoryorder='total ascending')  # Ensures correct descending order
)

# Display the figure
fig.show()

### Pull Projected Team Lineups 

In [ ]:
team_name = 'Chelsea'
# Step 1: Filter players based on the team name
team_players_df = player_statistics[player_statistics['team_name'] == team_name]
print(team_players_df[['player', 'position_abbrv', 'chance_of_playing_next_round']].head(20))

In [ ]:
# Example Usage
lineups_df = scrape_rotowire_lineups()
print(lineups_df.head(22))

In [ ]:
# Example usage
team_name = 'Arsenal'
player_data = lineups_df[lineups_df['Team'] == team_name]

plot_soccer_field(player_data, team_name)


### Import FPL Classical League Data

In [ ]:
## Set needed ID variables
brandon_classical_team_id = 6005298

# Set other variables
gameweek = 3

# Set the API endpoints
classic_gw_endpoint = f'https://fantasy.premierleague.com/api/entry/{brandon_classical_team_id}/event/{gameweek}/picks/'
classic_history_endpoint = f'https://fantasy.premierleague.com/api/entry/{brandon_classical_team_id}/history/'

#### FPL Classical Team History Endpoint

This endpoint gives summary information for my Classical League FPL team by gameweek. Key stats from this endpoint include:
- Gameweek Points
- Gameweek Rank
- Gameweek Points on Bench
- Historical Season Performances (Points & Rank)

In [ ]:
# Get the data from the endpoint
data = requests.get(classic_gw_endpoint)
# extracting data in json format
data_json = data.json()
# Pretty print with indentation
pretty_json = json.dumps(data_json, indent=4)
print(pretty_json[:1000])

#### FPL Classical Game Status Endpoint 

This endpoint gives detailed information for my Classical League FPL team by gameweek, including my lineups, points, who was captained, and whether and automatic chips were used. Some other important stats from this endpoint include:
- Total Points
- Rank
- Percentile Rank
- Bank
- Team Value
- Gameweek Transfers
- Points on Bench

In [ ]:
# Get the data from the endpoint
data = requests.get(classic_history_endpoint)
# extracting data in json format
data_json = data.json()
# Pretty print with indentation
pretty_json = json.dumps(data_json, indent=4)
print(pretty_json[:1000])

### Import FPL Draft League Data

In [ ]:
# Get the data from the endpoint
data = requests.get(draft_team_endpoint)
# extracting data in json format
data_json = data.json()
# Pretty print with indentation
pretty_json = json.dumps(data_json, indent=4)
print(pretty_json[:1000])

#### FPL Draft League Endpoint 

This endpoint gives information on the FPL Draft League, including the league name, format, and teams. Some key information from this endpoint include:
- League Name (league >> name)
- Scoring Format (league >> scoring)
- Draft Date (league >> draft_dt)
- Team Names (league_entries >> entry_name)
- Team IDs (league_entries >> entry_id)
- Player Names (league_entries >> player_first_name + player_last_name)
- Player Waiver Picks (league_entries >> waiver_pick)

In [ ]:
# Get the data from the endpoint
data = requests.get(draft_league_endpoint)
# extracting data in json format
data_json = data.json()
# Pretty print with indentation
pretty_json = json.dumps(data_json, indent=4)
print(pretty_json[:1000])

#### FPL Draft Fixtures Endpoint

In [ ]:
# Get the data from the endpoint
data = requests.get(draft_fixtures_endpoint)
print(data.text)
# extracting data in json format
data_json = data.json()
# Pretty print with indentation
pretty_json = json.dumps(data_json, indent=4)

#### FPL Draft Game Status Endpoint

This endpoint gives basic information on the gameweek. This include key stats such as:
- Current Gameweek (current_event)
- Whether Waivers have been processed (waivers_processed)

In [ ]:
# Get the data from the endpoint
data = requests.get(game_status_endpoint)
# extracting data in json format
data_json = data.json()
# Pretty print with indentation
pretty_json = json.dumps(data_json, indent=4)
print(pretty_json)

#### FPL Draft Picks Endpoint

This endpoint gives information on the FPL draft picks made by each team.

In [ ]:
# Get the data from the endpoint
data = requests.get(draft_picks_endpoint)
# extracting data in json format
data_json = data.json()
# Pretty print with indentation
pretty_json = json.dumps(data_json, indent=4)
print(pretty_json)

#### FPL Draft Waivers Endpoint

This endpoint gives information on all the waiver picks made by the league throughout the year. The key details of this endpoint include:
- Gameweek (transactions >> event)
- Team (transactions >> entry)
- Player Added (transactions >> element_in) # Note: this is just the player_id
- Player Dropped (transactions >> element_out) # Note: this is just the player_id
- Waiver Order (transactions >> priority)
- Waiver Result (transactions >> result) 
    - "a" = accepted
    - "di" = denied - player has already been taken
    - "do" = denied - too many players on your team
- Waiver Type (transactions >> kind)
    - "w" = waiver
    - "f" = free agent

In [ ]:
# Get the data from the endpoint
data = requests.get(waivers_endpoint)
# extracting data in json format
data_json = data.json()
# Pretty print with indentation
pretty_json = json.dumps(data_json, indent=4)
print(pretty_json)

#### FPL Draft Team Selection Endpoint

This endpoint gives informations on the starting lineup of an FPL Draft team. The key piece of information from this endpoint is the "position" under the "picks" key, which is where we can determine if a player was started. A position of 1-11 is a starting player and a position of 12-15 is a benched player. This also gives information on whether a player was a "captain", but that is irrelevant for our Draft settings. 

In [ ]:
# Get the data from the endpoint
data = requests.get(draft_gw_lineups_endpoint)
# extracting data in json format
data_json = data.json()
# Pretty print with indentation
pretty_json = json.dumps(data_json, indent=4)
print(pretty_json[:1000])

### FPL Draft Team Analysis

Want to create some code to perform an analysis on each of the team's in the league. This should include a comparitive analysis of their top performers, positional weaknesses, fixture results and schedule, and head to head league results.

In [ ]:
# Get mapping of team IDs from league
league_url = f"https://draft.premierleague.com/api/league/{brandon_draft_league_id}/details"
league_data = requests.get(league_url).json()

# Map team names to their entry IDs
team_map = {entry['entry_name']: entry['entry_id'] for entry in league_data['league_entries']}

team_name = 'Stoned Squirrels'
team_id = team_map[team_name]
print(f"Team ID: {team_id}")
gameweek = 10
# Fetch the team's lineup for the given gameweek
lineup_url = f"https://draft.premierleague.com/api/entry/{team_id}/event/{gameweek}"
lineup_data = requests.get(lineup_url).json()

# Fetch player mappings
player_mapping = get_fpl_player_mapping()  # Function should return {player_id: {'Player': 'Name', 'Team': 'XYZ', 'Position': 'G/D/M/F'}}
print(player_mapping)
# Separate starters and bench
starters = []
bench = []

In [ ]:
for pick in lineup_data["picks"]:
    player_id = pick["element"]
    is_starter = pick["position"] <= 11  # Positions 1-11 are starters

    player_info = player_mapping.get(player_id, {"Player": f"Unknown ({player_id})", "Team": "Unknown", "Position": "Unknown"})
    
    player_entry = {
        "Player": player_info["Player"],
        "Team": player_info["Team"],
        "Position": player_info["Position"]
    }

    if is_starter:
        starters.append(player_entry)
    else:
        bench.append(player_entry)

# Convert to DataFrame for better display
starters_df = pd.DataFrame(starters)
bench_df = pd.DataFrame(bench)

print(f"**{team_name} - Gameweek {gameweek} Lineup**")
print("\n**Starting XI:**")
print(starters_df)

print("\n**Bench:**")
print(bench_df)


#### FPL Draft Team Top Performers

In [88]:
# Fetch player and team mappings
player_map = get_fpl_player_mapping()
team_map = get_league_entries(brandon_draft_league_id)

# Initialize the team composition with the initial draft picks
draft_picks = get_starting_team_composition(brandon_draft_league_id)
team_composition = set(draft_picks.get(brandon_draft_team_id, {}).get('players', []))

# Apply relevant transactions to the team composition
transactions = get_waiver_transactions_up_to_gameweek(brandon_draft_league_id, gameweek)
for tx in transactions:
    if tx['entry'] == brandon_draft_team_id and tx['result'] == 'a':
        # Convert player IDs to names using player_map
        player_in = player_map.get(tx['element_in'], {}).get('Player', f"Unknown ({tx['element_in']})")
        player_out = player_map.get(tx['element_out'], {}).get('Player', f"Unknown ({tx['element_out']})")

        # Update the team composition
        team_composition.remove(player_out)
        team_composition.add(player_in)

# Convert the team composition to a DataFrame with player details
player_data = [
    player_map.get(player_id, {'Player': player_name, 'Team': 'Unknown', 'Position': 'Unknown'})
    for player_name, player_id in [(player, next((k for k, v in player_map.items() if v['Player'] == player), None))
                                   for player in team_composition]
]
print(player_data)
fpl_players_df = pd.DataFrame(player_data)

[{'Player': 'Pedro Porro', 'Web_Name': None, 'Team': 'TOT', 'Position': 'D'}, {'Player': 'Morgan Gibbs-White', 'Web_Name': 'Gibbs-White', 'Team': 'NFO', 'Position': 'M'}, {'Player': 'Rico Lewis', 'Web_Name': 'Lewis', 'Team': 'MCI', 'Position': 'D'}, {'Player': 'Morgan Rogers', 'Web_Name': 'Rogers', 'Team': 'AVL', 'Position': 'M'}, {'Player': 'Jadon Sancho', 'Web_Name': 'Sancho', 'Team': 'CHE', 'Position': 'M'}, {'Player': 'Matz Sels', 'Web_Name': 'Sels', 'Team': 'NFO', 'Position': 'G'}, {'Player': 'Dominic Calvert-Lewin', 'Web_Name': 'Calvert-Lewin', 'Team': 'EVE', 'Position': 'F'}, {'Player': 'Malo Gusto', 'Web_Name': 'Gusto', 'Team': 'CHE', 'Position': 'D'}, {'Player': 'Son Heung-min', 'Web_Name': 'Son', 'Team': 'TOT', 'Position': 'M'}, {'Player': 'Jørgen Strand Larsen', 'Web_Name': 'Strand Larsen', 'Team': 'WOL', 'Position': 'F'}, {'Player': 'Gabriel dos Santos Magalhães', 'Web_Name': 'Gabriel', 'Team': 'ARS', 'Position': 'D'}, {'Player': 'Chris Wood', 'Web_Name': 'Wood', 'Team': 'N

In [89]:
# Define inputs
gameweek = 20

# Get FPL fantasy team player composition
team_df = get_team_composition_for_gameweek(brandon_draft_league_id, brandon_draft_team_id, gameweek)
team_df


,Player,Web_Name,Team,Position
0,Pedro Porro,None,TOT,D
1,Morgan Gibbs-White,Gibbs-White,NFO,M
2,Rico Lewis,Lewis,MCI,D
3,Morgan Rogers,Rogers,AVL,M
4,Jadon Sancho,Sancho,CHE,M
5,Matz Sels,Sels,NFO,G
6,Dominic Calvert-Lewin,Calvert-Lewin,EVE,F
7,Malo Gusto,Gusto,CHE,D
8,Son Heung-min,Son,TOT,M
9,Jørgen Strand Larsen,Strand Larsen,WOL,F


In [ ]:
## Merge the player list with their total points from the stats DataFrame

# Merge on player name (fuzzy matching logic can be used here if needed)
team_points_df = pd.merge(team_df, player_stats_df[['player', 'total_points']], 
                          left_on='Player', right_on='player', how='left')

# Sort by total points
team_points_df = team_points_df.sort_values(by='total_points', ascending=False)

# Select relevant columns
team_points_df = team_points_df[['Player', 'Team', 'Position', 'total_points']].rename(
    columns={'total_points': 'Total Points'}
).reset_index(drop=True)

# Display top N
top_n = 10
print(team_points_df.head(top_n))

In [ ]:

# Filter by team
if team_name:
    filtered_df = player_statistics[player_statistics['team_name'] == team_name]
else:
    filtered_df = player_stats_df[player_stats_df['team'] == team_id]

# Ensure total_points is numeric
filtered_df['total_points'] = pd.to_numeric(filtered_df['total_points'], errors='coerce').fillna(0)

# Sort and return top N
top_players = (
    filtered_df
    .sort_values(by='total_points', ascending=False)
    .loc[:, ['player', 'position_abbrv', 'team_name', 'total_points']]
    .head(top_n)
    .rename(columns={
        'player': 'Player',
        'position_abbrv': 'Position',
        'team_name': 'Team',
        'total_points': 'Total Points'
    })
    .reset_index(drop=True)
)

return top_players


#### FPL Draft Head to Head League Results

In [ ]:
# Extract league entries and match history
league_entries = league_response['league_entries']
fixtures = league_response['matches']

# Create a dictionary mapping team IDs to team names
team_names = {entry['id']: entry['entry_name'] for entry in league_entries}

selected_team_name = 'Stoned Squirrels'

# Get the team ID for the selected team
selected_team_id = next((id for id, name in team_names.items() if name == selected_team_name), None)

# if selected_team_id is None:
#     raise ValueError(f"Team '{selected_team_name}' not found in the league.")

In [ ]:
# Initialize a dictionary to store head-to-head records
head_to_head_records = {team: {"W": 0, "D": 0, "L": 0, "Total Games": 0} for team in team_names.values() if team != selected_team_name}

# Iterate through fixtures and update head-to-head records
for fixture in fixtures:
    team1_id = fixture['league_entry_1']
    team2_id = fixture['league_entry_2']
    team1_score = fixture['league_entry_1_points']
    team2_score = fixture['league_entry_2_points']

    if team1_score == 0 and team2_score == 0:  # Skip if match hasn't been played
        continue

    # Get team names
    team1_name = team_names.get(team1_id)
    team2_name = team_names.get(team2_id)

    # Determine if the selected team is in this fixture
    if team1_id == selected_team_id or team2_id == selected_team_id:
        opponent_name = team2_name if team1_id == selected_team_id else team1_name
        selected_team_score = team1_score if team1_id == selected_team_id else team2_score
        opponent_score = team2_score if team1_id == selected_team_id else team1_score

        # Determine result
        if selected_team_score > opponent_score:
            head_to_head_records[opponent_name]["W"] += 1
        elif selected_team_score < opponent_score:
            head_to_head_records[opponent_name]["L"] += 1
        else:
            head_to_head_records[opponent_name]["D"] += 1
        
        head_to_head_records[opponent_name]["Total Games"] += 1

# Convert head-to-head records to a DataFrame
h2h_df = pd.DataFrame([
    {"Opponent": opponent, "Wins": record["W"], "Draws": record["D"], "Losses": record["L"], 
     "Total Games": record["Total Games"],
     "Win %": (record["W"] / record["Total Games"] * 100) if record["Total Games"] > 0 else 0,
     "Draw %": (record["D"] / record["Total Games"] * 100) if record["Total Games"] > 0 else 0,
     "Loss %": (record["L"] / record["Total Games"] * 100) if record["Total Games"] > 0 else 0
    }
    for opponent, record in head_to_head_records.items()
])

# **Sorting the DataFrame**
h2h_df = h2h_df.sort_values(
    by=["Win %", "Draw %", "Total Games"], 
    ascending=[False, False, False]  # Sort by win % (desc), then draw % (desc), then total games (desc)
)

# Print the head-to-head record
print(f"Chosen Team: {selected_team_name}")
for _, row in h2h_df.iterrows():
    print(f"{row['Wins']}-{row['Draws']}-{row['Losses']} vs {row['Opponent']}")

In [ ]:
h2h_df

In [ ]:
h2h_df = h2h_df[h2h_df["Total Games"] > 0]  # Remove teams with no matches played

fig = go.Figure()

# Add stacked bars for wins, draws, and losses
for result, color, text_col in zip(
    ["Win %", "Draw %", "Loss %"], 
    ["green", "white", "red"], 
    ["Wins", "Draws", "Losses"]
):
    fig.add_trace(go.Bar(
        y=h2h_df["Opponent"], 
        x=h2h_df[result], 
        name=text_col,
        orientation="h", 
        marker=dict(color=color), 
        text=h2h_df[result].apply(lambda x: f"{x:.1f}%" if x > 0 else ""),
        textposition="inside",
        hoverinfo="text",
        hovertext=[
            f"<b>Wins</b> = {w}<br><b>Losses</b> = {l}<br><b>Draws</b> = {d}"
            for w, l, d in zip(h2h_df["Wins"], h2h_df["Losses"], h2h_df["Draws"])
        ]
    ))

fig.update_layout(
    title_text=f"Head-to-Head Record: {selected_team_name}",
    barmode="stack",
    xaxis=dict(title="Win/Draw/Loss %", tickmode="linear", dtick=20, range=[0, 100]),
    yaxis=dict(title="Opponent"),
    legend=dict(title="Result", traceorder="normal"),
    plot_bgcolor="rgba(0,0,0,0)",
)

fig.show()

### Import FPL Player Data 

In [ ]:
## Set needed ID variables
brandon_draft_league_id = 49249
brandon_draft_team_id = 189880
gameweek = get_current_gameweek()

# Draft League API URLs
player_stats_endpoint = 'https://draft.premierleague.com/api/bootstrap-static'
player_gw_stats_endpoint = f'https://draft.premierleague.com/api/event/{gameweek}/live'
player_owner_endpoint = f'https://draft.premierleague.com/api/league/{brandon_draft_league_id}/element-status'

#### FPL Draft Player Stats Endpoint

This endpoint gives information on all FPL player's season stats. The key pieces of data that can be extracted from this endpoint include: 
- Player ID (elements >> id)
- Player Name (elements >> player_first_name + player_second_name)
- Player Team (elements >> team)
- Assists (elements >> id)
- Bonus (elements >> id)
- Clean Sheets (elements >> id)
- Goals Scored (elements >> id)
- Goals Conceded (elements >> id)
- Penalties Missed (elements >> penalties_missed)
- Penalties Saved (elements >> penalties_saved)
- Expected Goals (elements >> expected_goals)
- Expected Assists (elements >> expected_assists)
- Expected Goal Involvements (elements >> expected_goal_involvements)
- Expected Goals Conceded (elements >> expected_goals_conceded)
- Minutes (elements >> minutes)
- Yellow Cards (elements >> yellow_cards)
- Red Cards (elements >> red_cards)
- Starts (elements >> start)
- Chance of Player This Round (elements >> chance_of_playing_this_round)
- Chance of Player Next Round (elements >> chance_of_playing_next_round)
- Points Per Game (elements >> points_per_game)
- Player News (elements >> news)

In [ ]:
# Get the data from the endpoint
data = requests.get(player_stats_endpoint)
# extracting data in json format
data_json = data.json()
# Pretty print with indentation
pretty_json = json.dumps(data_json, indent=4)
print(pretty_json)

#### FPL Draft Player Stats Endpoint

Gives detailed information on an FPL player's gameweek stats and the resulting FPL points.
- Player ID (elements >> id)
- Minutes Played (elements >> id # >> stats >> minutes)
- Total Points (elements >> id # >> stats >> total_points)
- Goals (elements >> id # >> stats >> goals_scored)
- Own Goals (elements >> id # >> stats >> own_goals)
- Assists (elements >> id # >> stats >> assists)
- Bonus (elements >> id # >> stats >> bonus)
- Clean Sheets (elements >> id # >> stats >> clean_sheets)
- Goals Conceded (elements >> id # >> stats >> goals_conceded)
- Penalties Missed (elements >> id # >> stats >> penalties_missed)
- Penalties Saved (elements >> id # >> stats >> penalties_saved)
- Expected Goals (elements >> id # >> stats >> expected_goals)
- Expected Assists (elements >> id # >> stats >> expected_assists)
- Expected Goal Involvements (elements >> id # >> stats >> expected_goal_involvements)
- Expected Goals Conceded (elements >> id # >> stats >> expected_goals_conceded)
- Yellow Cards (elements >> id # >> stats >> yellow_cards)
- Red Cards (elements >> id # >> stats >> red_cards)

In [ ]:
# Get the data from the endpoint
data = requests.get(player_gw_stats_endpoint)
# extracting data in json format
data_json = data.json()
# Pretty print with indentation
pretty_json = json.dumps(data_json, indent=4)
print(pretty_json)

In [ ]:
# Get the data from the endpoint
data = requests.get(player_owner_endpoint)
# extracting data in json format
data_json = data.json()
# Pretty print with indentation
pretty_json = json.dumps(data_json, indent=4)
print(pretty_json)


### Import FPL Table Standings 

In [95]:
get_fpl_draft_league_standings(brandon_draft_league_id, show_advanced_stats=False)

,Team,Player,W,D,L,PF,PA,Pts
Rank,,,,,,,,
1,COYS,John Churchill,23,0,9,1481,1223,69
2,Stoned Squirrels,Brandon Croarkin,20,0,12,1455,1289,60
3,More Salah,Nick Adams,19,1,12,1468,1322,58
4,5 goals vs. PA FC,Robert Miller,18,1,13,1316,1190,55
5,Kekambas FC,Mitch Balesi,18,0,14,1205,1237,54
6,VA Tractor Boyz,Quan Phan,16,0,16,1212,1213,48
7,Chappy’s Goats,Tim C,14,0,18,1174,1317,42
8,Wilson,Andrew Wilson,12,1,19,1251,1368,37
9,Real Tyler,Tyler Warren,9,2,21,1135,1318,29


In [96]:
get_luck_adjusted_league_standings(brandon_draft_league_id)

,Team,Avg_GW_Rank,Total_Points,Avg_Score,Fair_Rank,Actual_Rank,Luck_Index
6,Stoned Squirrels,11.75,60,45.47,1,2.0,-1.0
4,More Salah,12.03,58,45.88,2,3.0,-1.0
1,COYS,12.16,69,46.28,3,1.0,2.0
0,5 goals vs. PA FC,14.84,55,41.12,4,4.0,0.0
9,Wilson,16.94,37,39.09,5,8.0,-3.0
8,VA Tractor Boyz,17.47,48,37.88,6,6.0,0.0
3,Kekambas FC,17.62,54,37.66,7,5.0,2.0
2,Chappy’s Goats,18.69,42,36.69,8,7.0,1.0
7,Tofu’s Boys,19.31,24,35.94,9,10.0,-1.0
5,Real Tyler,20.38,29,35.47,10,9.0,1.0


In [145]:

# Draft League API URLs
league_url = f"https://draft.premierleague.com/api/league/{brandon_draft_league_id}/details"

# Get league details, fixtures, and team details
league_response = requests.get(league_url).json()

# Extract the standings and league_entries sections of the JSON
fixtures = league_response['matches']
league_entries = league_response['league_entries']
# Create a dictionary mapping entry_ids to team names
team_names = {entry['id']: entry['entry_name'] for entry in league_entries}

# Create a list to hold fixture details
fixtures_list = []

for fixture in fixtures:
    gameweek = fixture['event']
    team1_id = fixture['league_entry_1']
    team2_id = fixture['league_entry_2']
    team1_name = team_names.get(team1_id)
    team2_name = team_names.get(team2_id)
    team1_score = fixture['league_entry_1_points']
    team2_score = fixture['league_entry_2_points']

    # Filter out games that have not been played yet
    if (team1_score != 0) & (team2_score != 0):
        fixtures_list.append({
            "Gameweek": gameweek,
            "Team 1": team1_name,
            "Team 1 Score": team1_score,
            "Team 2": team2_name,
            "Team 2 Score": team2_score,
        })

# Convert fixtures_list to dataframe
fixtures_df = pd.DataFrame(fixtures_list)

# Create a long-format dataframe with each team's score and result for the week
long_format_df = pd.concat([
    fixtures_df[['Team 1', 'Team 1 Score', 'Gameweek']].rename(columns={'Team 1': 'Team', 'Team 1 Score': 'Score'}),
    fixtures_df[['Team 2', 'Team 2 Score', 'Gameweek']].rename(columns={'Team 2': 'Team', 'Team 2 Score': 'Score'})
])

# Sort by Gameweek for clarity
long_format_df = long_format_df.sort_values(by=["Gameweek"]).reset_index(drop=True)

# Calculate the average score per gameweek
avg_scores = long_format_df.groupby("Gameweek")["Score"].transform("mean")

# Assign result based on comparison to gameweek average
long_format_df["Result"] = long_format_df["Score"].apply(lambda x: 'W' if x > avg_scores[long_format_df["Score"] == x].iloc[0]
                                 else 'D' if x == avg_scores[long_format_df["Score"] == x].iloc[0]
                                 else 'L')

# Calculate W/L/T counts per team
result_counts = pd.crosstab(long_format_df['Team'], long_format_df['Result']).reset_index()

# Ensure all result columns are present
for col in ['W', 'D', 'L']:
    if col not in result_counts.columns:
        result_counts[col] = 0
        
# Calculate the rank for each team in each week based on their score
long_format_df['GW_Rank'] = long_format_df.groupby("Gameweek")["Score"].rank(ascending=False, method='min').astype(int)

# Calculate the actual results for each team based on their results (W, L, D)
long_format_df['Points'] = long_format_df['Result'].apply(lambda x: 3 if x == 'W' else (1 if x == 'D' else 0))

# Calculate the average weekly rank (Fair_Rank) and total points (actual standings) for each team
luck_adjusted_df = long_format_df.groupby('Team').agg(
    Avg_GW_Rank=('GW_Rank', 'mean'),  # The "fair" rank based on average weekly score rank
    Total_Points=('Points', 'sum'),   # The actual points based on match results
    Avg_Score=('Score', 'mean')       # The average score across all weeks
).reset_index()

# Merge luck_adjusted_df with result_counts
luck_adjusted_df = luck_adjusted_df.merge(result_counts[['Team', 'W', 'D', 'L']], on='Team', how='left')

# Create the adjusted standings by ranking teams based on Avg_Week_Rank
luck_adjusted_df['Fair_Rank'] = luck_adjusted_df['Avg_GW_Rank'].rank(ascending=True, method='min').astype(int)

# Sort by Fair_Rank to get the adjusted standings
luck_adjusted_df = luck_adjusted_df.sort_values(by='Fair_Rank')

# Format number values
luck_adjusted_df['Avg_GW_Rank'] = round(luck_adjusted_df['Avg_GW_Rank'], 2)
luck_adjusted_df['Avg_GW_Score'] = round(luck_adjusted_df['Avg_Score'], 2)
# Format final results
luck_adjusted_df = luck_adjusted_df[['Fair_Rank', 'Team', 'W', 'D', 'L', 'Avg_GW_Score', 'Avg_GW_Rank', 'Total_Points']]
# Set the index
luck_adjusted_df.set_index('Fair_Rank', inplace=True)
print(luck_adjusted_df)

                        Team   W  D   L  Avg_GW_Score  Avg_GW_Rank  \
Fair_Rank                                                            
1           Stoned Squirrels  22  0  10         45.47         4.00   
2                       COYS  22  0  10         46.28         4.09   
2                 More Salah  19  0  13         45.88         4.09   
4          5 goals vs. PA FC  17  0  15         41.12         5.22   
5                     Wilson  17  0  15         39.09         5.66   
6            VA Tractor Boyz  16  0  16         37.88         5.75   
7                Kekambas FC  16  0  16         37.66         6.00   
8             Chappy’s Goats  11  0  21         36.69         6.28   
9                 Real Tyler  10  0  22         35.47         6.41   
10               Tofu’s Boys  11  0  21         35.94         6.50   

           Total_Points  
Fair_Rank                
1                    66  
2                    66  
2                    57  
4                    51  
5  

In [ ]:
get_league_standings(brandon_draft_league_id, show_advanced_stats=True)

In [ ]:
plot_team_points_over_time(brandon_draft_league_id)

In [ ]:
plot_league_points_over_time(brandon_draft_league_id)

### Pull FPL Draft Fixture Results 

In [ ]:
gameweek = 4
show_fixture_results(brandon_draft_league_id, gameweek)

###  FPL Draft Gameweek Fixtures

In [ ]:
# Step 1: Get the current gameweek
current_gameweek = get_current_gameweek()

# Step 2: Fetch league details
league_url = f"https://draft.premierleague.com/api/league/{brandon_draft_league_id}/details"
league_response = requests.get(league_url).json()

# Extract the standings and league_entries sections of the JSON
fixtures = league_response['matches']
league_entries = league_response['league_entries']

# Create a dictionary mapping entry_ids to team names and managers
team_info = {entry['id']: (entry['entry_name'], entry['player_first_name'] + ' ' + entry['player_last_name']) 
             for entry in league_entries}

# Set the current_gameweek
current_gameweek = get_current_gameweek()

# Filter fixtures for the current gameweek
gameweek_fixtures = [f for f in fixtures if f['event'] == current_gameweek]

# Print the fixtures
for fixture in gameweek_fixtures:
    team1_id = fixture['league_entry_1']
    team2_id = fixture['league_entry_2']
    # Get team and manager info with fallbacks
    team1_name, team1_manager = team_info.get(team1_id, ("Unknown Team", "Unknown Manager"))
    team2_name, team2_manager = team_info.get(team2_id, ("Unknown Team", "Unknown Manager"))

    print(f"{team1_name} ({team1_manager}) v {team2_name} ({team2_manager})")

In [ ]:
# Find the fixtures for the current gameweek
gameweek_fixtures = get_gameweek_fixtures(brandon_draft_league_id, get_current_gameweek())
print(gameweek_fixtures)

###  FPL Draft Fixture Projections

In [ ]:
team_id = 189880
get_team_composition_for_gameweek(brandon_draft_league_id, team_id, current_gameweek)

In [ ]:
fpl_player_projections

In [ ]:
team1_df, team2_df, team1_name, team2_name = analyze_fixture_projections(gameweek_fixtures[0], 
                                                                         brandon_draft_league_id, 
                                                                         fpl_player_projections)
print(team1_df)
print(team2_df)

## Waiver Team Handling

We want to be able to update these teams above to reflect the weekly transactions made by a team. 

In [ ]:
# FPL Draft picks endpoint
draft_url = f"https://draft.premierleague.com/api/draft/{brandon_draft_league_id}/choices"
# Pull and format the response
response = requests.get(draft_url)
draft_data = response.json()

draft_picks = {}

# Parse the draft picks
for choice in draft_data['choices']:
    team_name = choice['entry_name']
    player_id = choice['element']

    if team_name not in draft_picks:
        draft_picks[team_name] = []

    draft_picks[team_name].append(player_id)

In [ ]:
# Draft League API URLs
transactions_url = f"https://draft.premierleague.com/api/draft/league/{brandon_draft_league_id}/transactions"

# Get transaction details
transactions_response = requests.get(transactions_url).json()
print(transactions_response)

In [ ]:
# URL to fetch the transactions
transaction_url = f"https://draft.premierleague.com/api/draft/league/{brandon_draft_league_id}/transactions"
transaction_response = requests.get(transaction_url)
transaction_data = transaction_response.json()

# Get the player data
player_url = "https://draft.premierleague.com/api/bootstrap-static"
player_response = requests.get(player_url)
player_data = player_response.json()

# Fetch league details to get team names
league_url = f"https://draft.premierleague.com/api/league/{brandon_draft_league_id}/details"
league_response = requests.get(league_url)
league_data = league_response.json()

# Create a dictionary mapping entry IDs to team names
team_dict = {entry['entry_id']: entry['entry_name'] for entry in league_data['league_entries']}

# Create a dictionary mapping player IDs to full player names (first_name + second_name)
player_dict = {player['id']: f"{player['first_name']} {player['second_name']}" for player in player_data['elements']}

# Loop through each transaction and print only approved transactions
print("Approved Waiver Transactions:")
for transaction in transaction_data['transactions']:
    if transaction['result'] == 'a':  # Only process approved transactions
        added_player_id = transaction.get('element_in')
        removed_player_id = transaction.get('element_out')
        entry_id = transaction.get('entry')

        # Get player names from the player_dict
        added_player_name = player_dict.get(added_player_id, "Unknown Player")
        removed_player_name = player_dict.get(removed_player_id, "Unknown Player")

        # Get the team name from the team_dict
        team_name = team_dict.get(entry_id, "Unknown Team")

        # Print the transaction details
        print(f"Transaction ID: {transaction['id']}. Gameweek: {transaction['event']}")
        print(f"  Team: {team_name} (Team ID: {entry_id})")
        print(f"  Added: {added_player_name} (Player ID: {added_player_id})")
        print(f"  Removed: {removed_player_name} (Player ID: {removed_player_id})")

        print("-" * 30)

## Team Composition

In [92]:
# Example Usage
league_id = 49249
team_name = "Stoned Squirrels"
gameweek = 9

try:
    team_composition_df = get_team_composition_for_gameweek(league_id, team_name, gameweek)
    print(f"Team composition for {team_name} in Gameweek {gameweek}:\n{team_composition_df}")
except ValueError as e:
    print(e)


Team composition for Stoned Squirrels in Gameweek 9:
Empty DataFrame
Columns: []
Index: []


In [93]:
team_name = "Tofu's Boys"
print(team_name)
# Fetch league entries to map team names to IDs
team_map = get_league_entries(brandon_draft_league_id)
print("Team Map: ", team_map.items())

# Search for the team ID by team name
team_id = next((id for id, name in team_map.items() if name == team_name), None)
print(team_id)
if team_id is None:
    print(f"Team '{team_name}' not found in the league.")



Tofu's Boys
Team Map:  dict_items([(189880, 'Stoned Squirrels'), (189929, 'Chappy’s Goats'), (189959, 'Tofu’s Boys'), (190444, 'More Salah'), (266040, 'Kekambas FC'), (296475, 'COYS'), (298333, 'Wilson'), (322437, '5 goals vs. PA FC'), (338004, 'VA Tractor Boyz'), (338058, 'Real Tyler')])
None
Team 'Tofu's Boys' not found in the league.


In [ ]:
fpl_player_projections.head()

In [ ]:
def get_league_player_ownership(league_id):
    """
    Fetches the player ownership in the league for the current gameweek and groups the players by team and position.

    Parameters:
    - league_id: The ID of the FPL Draft league.

    Returns:
    - A dictionary structured as:
      {'Team Name': {'G': [list of goalkeepers], 'D': [list of defenders], 'M': [list of midfielders], 'F': [list of forwards]}}
    """
    # Endpoint URLs
    element_status_url = f"https://draft.premierleague.com/api/league/{league_id}/element-status"
    league_details_url = f"https://draft.premierleague.com/api/league/{league_id}/details"

    # Fetch the element status and league details
    element_status = requests.get(element_status_url).json()['element_status']
    league_details = requests.get(league_details_url).json()

    # Fetch player and owner mappings
    player_map = get_fpl_player_mapping()  # {player_id: {'Player': 'Name', 'Team': 'XYZ', 'Position': 'M'}, ...}
    owner_map = {entry['id']: entry['entry_name'] for entry in league_details['league_entries']}  # {owner_id: 'Team Name'}

    # Initialize a dictionary to group players by team and position
    league_ownership = defaultdict(lambda: {'G': [], 'D': [], 'M': [], 'F': []})

    for status in element_status:
        player_id = status['element']
        owner_id = status['owner']
        if owner_id is None:  # Skip players without an owner
            continue

        # Convert player ID to name and position
        player_info = player_map.get(player_id, {'Player': f"Unknown ({player_id})", 'Position': 'Unknown'})
        player_name = player_info['Player']
        player_position = player_info['Position']

        # Convert owner ID to team name
        team_name = owner_map.get(owner_id, f"Unknown Team ({owner_id})")

        # Add the player to the appropriate team and position group
        if player_position in ['G', 'D', 'M', 'F']:  # Valid positions
            league_ownership[team_name][player_position].append(player_name)

    return dict(league_ownership)

In [ ]:
# Endpoint URLs
element_status_url = f"https://draft.premierleague.com/api/league/{league_id}/element-status"
league_details_url = f"https://draft.premierleague.com/api/league/{league_id}/details"

# Fetch the element status and league details
element_status = requests.get(element_status_url).json()['element_status']
league_details = requests.get(league_details_url).json()

# Fetch player and owner mappings
player_map = get_fpl_player_mapping()  # {player_id: {'Player': 'Name', 'Team': 'XYZ', 'Position': 'M'}, ...}
owner_map = {entry['entry_id']: entry['entry_name'] for entry in league_details['league_entries']}  # {owner_id: 'Team Name'}

In [ ]:
get_league_player_ownership(brandon_draft_league_id)

In [ ]:
merged_df = merge_fpl_and_projections(team_composition_df, player_projections)
print(merged_df)


## Team Optimizer 

## Waiver Optimizer 

In [ ]:
# Pull FPL player rankings from Rotowire
player_rankings = get_rotowire_player_projections(url, limit=None)

# Get the league player dict for the current gameweek
league_player_dict = get_league_player_ownership(brandon_draft_league_id)
print(league_player_dict)

In [ ]:

# Endpoint to fetch fixture data
fixtures_url = "https://draft.premierleague.com/api/bootstrap-static"

# Fetch data from the API
response = requests.get(fixtures_url)
response.raise_for_status()
data = response.json()

# Extract relevant sections
if 'fixtures' not in data or 'teams' not in data:
    raise ValueError("Unexpected API response structure. Missing 'fixtures' or 'teams'.")

fixtures = data['fixtures']
teams = {team['id']: team['short_name'] for team in data['teams']}

# List to hold structured fixture data
fixtures_data = []

# Iterate through gameweeks and fixtures
for gameweek, gameweek_fixtures in fixtures.items():
    for fixture in gameweek_fixtures:
        print(fixture)
        # Extract relevant fixture details
        fixtures_data.append({
            "Gameweek": int(gameweek),
            "Home Team": teams.get(fixture['team_h'], f"Unknown ({fixture['team_h']})"),
            "Away Team": teams.get(fixture['team_a'], f"Unknown ({fixture['team_a']})"),
            "Kickoff Time": fixture.get('kickoff_time'),
            "Home Team Difficulty": fixture.get('team_h_difficulty', "N/A"),
            "Away Team Difficulty": fixture.get('team_a_difficulty', "N/A")
        })

# Convert to a DataFrame
fixtures_df = pd.DataFrame(fixtures_data)

# Sort by gameweek and kickoff time for better readability
fixtures_df.sort_values(by=['Gameweek', 'Kickoff Time'], inplace=True)


In [ ]:
# Pull the top available waivers based on the FPL player rankings and slider value
top_available_players = find_top_waivers(player_rankings, league_player_dict, 100)
print(top_available_players)